In [2]:
from bertopic import BERTopic
from scipy.cluster import hierarchy as sch
import pandas as pd
import logging
from sklearn.metrics.pairwise import cosine_distances
from tqdm import tqdm
import numpy as np
from umap import UMAP
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
import re
import sys
sys.path.append('../../')
from utility.utility_functions import *

In [3]:
#  '\\\\Cm\\CM\\'

In [ ]:
df = pd.read_parquet('../../data/processed/05_avocado_cleaned_clustered_wip.parquet', engine='pyarrow')

,text
0,Bug 5539\r\nThe screen wizard now ignores the ...
1,"At 02:50 PM 3/15/2001 +0100, it seemed like Ma..."
2,"All,\r\nWe are still in a Nor'easter, but the ..."
3,Responding to Robert Sartin (07:02 AM 6/14/200...
4,I will try to get the real secure up at the fi...
...,...
128640,Checked in WinInetBrowser.java to fix a bug in...
128641,Just wanted to make sure that you know I did t...
128642,"In the end, you will always fail to think of e..."
128643,"Yes, it's valid for Europe, as well. Venk for..."


In [5]:
texts = df['text']

In [9]:
test = texts[texts.str.startswith('\\\\')]
testest = test.tolist()
testest.sort()
testest

['\\\\Cm\\Burn\\4.5GA_Germanium.x\\05-06-2002\\003\\fullinstaller',
 "\\\\Cm\\CM\\Build\\4.0GA_palladium.x\\04-01-2002\\136\\fullinstaller\\setup.exe\r\nWith this build, Full Installer seems working fine, but there are problems.\r\n1. You can not run OCA.  If you try to run OCA, from d:\\build\\bin\\runOCA.bat --> it just doesn't run\r\n2. If you try to write your own batch file to run OCA, since the schema.xml is not there, the Biogen Application won't run\r\n3. The convertutil.jar is not there in the build.  So we can not use Roopak's utility.\r\nSreenivas",
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\05-07-2002\\004\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\06-26-2002\\1090\\fullinstaller\\setup.exe\r\nHi Wilhan,\r\nInstaller fails to install the online.  Do you know how to proceed?',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\06-27-2002\\037\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\07-08-2002\\039\\fullinstaller',
 '\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\07-2

In [8]:
for t in test:
    print(repr(t))

"\\\\Cm\\CM\\Build\\4.0GA_palladium.x\\04-01-2002\\136\\fullinstaller\\setup.exe\r\nWith this build, Full Installer seems working fine, but there are problems.\r\n1. You can not run OCA.  If you try to run OCA, from d:\\build\\bin\\runOCA.bat --> it just doesn't run\r\n2. If you try to write your own batch file to run OCA, since the schema.xml is not there, the Biogen Application won't run\r\n3. The convertutil.jar is not there in the build.  So we can not use Roopak's utility.\r\nSreenivas"
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\10-03-2002\\132\\fullinstaller'
'\\\\Cm\\CM\\Build\\4.6GA_Neon.x\\12-25-2002\\043\\fullinstaller\r\nThe EMDS online tests have passed over this build.\r\nSreenivas'
'\\\\yuyu\\software\\jprobe\r\ncheck sno.txt for license key.\r\n-yuyu'
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\10-14-2002\\140\\fullinstaller'
'\\\\Cm\\CM\\Build\\4.5GA_Germanium.x\\09-30-2002\\129'
'\\\\Utilitysrv1\\home\\Marketing\\solutions_folder'
'\\\\utilitysrv1\\home\\SalesEngineering\\Accou

In [ ]:
removal_list = [
    '\r\n--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)',
    '\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering \t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w\r\nAvocadoIT, Your Business Everywhere !',
    '\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w',
    '--------------------------\r\nFind out how AvocadoIT can mobilize your business (www.avocadoit.com)',
    '\r\n--------------------------\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n-------------------------------------------------------------------\r\n** AvocadoIT, Your Business Everywhere **',
    '\r\n--------------------------\r\nAvocadoIT, Your Business Everywhere! (www.avocadoit.com)',
    '\r\n--------------------------\r\nEmpowered by AvocadoIT Technology (www.avocadoit.com)',
    '************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************',
    '\r\n--------------------------\r\nSent to you wirelessly.\r\nAndy Wong\r\n408.562.8131\r\nawong@avocadoit.com',
    '\r\nYour Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************',
    '\r\nEnabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8039\r\nMobile: (415) 845-6789\r\nFax: (408) 408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************',
    '\r\nEnabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************',
    '\r\n--------------------------\r\nChris Longstaffe\r\n(408) 205 7573\r\nSent using my AvocadoIT wireless device\r\nM-business ... M-powered by AvocadoIT!',
    '\r\nPeter Smialek\r\nRegional Sales Director\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile',
    '\r\nAvocadoIT, Your Business Everywhere !',
    '\r\nDirector of Sales, Northeast\r\nAvocadoIT, Inc.\r\n508-893-6095  Office\r\n508-893-0828  Fax\r\n508-740-9126  Mobile\r\njeff.newman@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nCraig Harper\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8175\r\nFax: 408.562.8100\r\ncraig.harper@avocadoit.com\r\nrm -Rf //taliban/bin/laden',
    '\r\n***********************************************************************\r\nRoopak Parikh\r\navocadoit.com\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 588-4057 x17\r\n**********************************************************************',
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com',
    '\r\n________________________\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com',
    '\r\nMatt\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nMatt DiMaria\r\nSVP, Marketing and Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Office\r\n(215) 778-3235  Mobile\r\nAvocadoIT, Your Business Everywhere! www.avocadoit.com',
    '\r\n---\r\nRicardo Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 829-4827 cell\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nBetty Chan-Bauza\r\nGlobal Alliance Manager\r\nAvocadoIT, Inc.\r\nDirect: 1.408.562.8192\r\nFax: 1.408-562.8100\r\nbchan@avocadoit.com\r\nwww.avocadoit.com',
    '\r\n---\r\nGeoff Vona \r\nAvocadoIT\r\nhttp://www.avocadoit.com',
    '\r\n---\r\nRicardo Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 829-4827 cell\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nLaura C. Henderson\r\nClient Development\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nCell: (408) 857.8529\r\nhttp://www.avocadoit.com',
    '\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nMatt DiMaria\r\nSVP, Marketing and Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '\r\nCraig Stewart\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, California 95054\r\nMobile:   510-409-7173\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    cstewart@avocadoit.com\r\nWeb:      www.avocadoit.com',
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n*New Address*\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\ndebbiemargulies@hotmail.com\r\nwww.avocadoit.com',
    '\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com \r\nYour Business Everywhere!',
    '\r\nCraig Stewart\r\nVice President\r\nGlobal Telecom Business Unit\r\nOffice: 408-562-8060\r\nFax: 408-562-8100\r\nwww.avocadoit.com',
    '\r\nSent wirelessly from AvocadoIT, Inc. \r\nPrakash Iyer\r\nFounder, CTO & Sr. Vice President, Engineering\r\nDirect: 408.562.8030 \r\npiyer@avocadoit.com \r\nAvocadoIT, Your Business Everywhere!',
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nElba Linscott\r\nSales Engineer\r\n408.562.8027 Direct\r\n408.891.8134 Cell \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nelba.linscott@avocadoit.com <mail',
    '\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: 408.857.8744\r\nFax: 408.562.8100 \r\ndoug.field@avocadoit.com\r\nPalm:  ep036@palm.net \r\nwww.avocadoit.com',
    '\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Inc. \r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nYour Business Everywhere!',
    '\r\nRon\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector, AvocadoITOnline\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************',
    '\r\n--------------------------------------\r\nJimmy Kim\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n4933 West Louise\r\nSuite 101\r\nSkokie, IL 60077\r\nOffice: 312-409-4919\r\nMobile: 312-405-3836\r\nFax: 810.279.4299\r\njimmy.kim@avocadoit.com\r\nwww.avocadoit.com',
    '\r\nTrish Lady\r\nVP, Business Development \r\nAvocadoIT, Inc.\r\n2211 N. 1st Street, San Jose, CA 95131\r\np:  408-562-8070  m:  415-505-3756   f:  408-562-8100  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?',
    '\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n*New Address* \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\ndebbiemargulies@hotmail.com \r\nwww.avocadoit.com',
    '\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nMobile:   408-891-8139\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    cstewart@avocadoit.com\r\nWeb:      www.avocadoit.com',
    '---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w',
    '\r\n---\r\nDan Baca \t\t\tdbaca@avocadoit.com\t\t408.562.8041\r\nDirector of Engineering\t\twww.avocadoit.com\t\t408.394.6114 m',
    '\r\nElisabeth V. Whaley\r\nAssit. to the VP of Prof. Srvcs.\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n----------------------------------\r\nBrad Seipp                \r\nSales Engineer          \r\n310.621.8201 W\r\nwww.avocadoit.com \r\nSent from my AvocadoIT \r\nWireless Handheld',
    '\r\n--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc .\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com'
    '\r\n________________________________________________\r\nRishi Manocha\r\nProject Manager\r\nEmail : rishi.manocha@avocadoit.com\r\nDirect: 408-562-8112\r\nMobile: 408-857-8746',
    '\r\n*******************************************************************\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Mail\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com \r\n*******************************************************************\r\n',
    '\r\n--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc .\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com',
    '\r\n________________________________________________\r\nRishi Manocha\r\nProject Manager\r\nEmail : rishi.manocha@avocadoit.com\r\nDirect: 408-562-8112\r\nMobile: 408-857-8746',
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.*\r\n3965 Freedom Circle\r\nSuite 1100\r\nSanta Clara, CA 95054\r\n*We've moved (as of July 1, 2003)\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com\r\nYour Business Everywhere!',
    "\r\nYour Business Everywhere\r\n-- \r\nAvocadoIT Europe GmbH \r\nKönigsallee 106 \r\n40215 Düsseldorf, Germany \r\n(p)     +49 (0) 211.30122-125 \r\n(f)     +49 (0) 211.30122-200 \r\n(m)     +49 (0) 171.3845255 \r\nmailwww.avocadoit.com",
    "\r\nBrett W. Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nLaura C. Henderson\r\nClient Development\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103 ext 8187\r\nDirect: (408) 562.8187\r\nCell: (408) 857.8529\r\nhttp://www.avocadoit.com",
    "\r\nChristopher Quinter\r\nAccount Manager\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\nOffice:  (267) 757-0736\r\nMobilizing solutions to reduce costs, increase revenue & improve productivity www.avocadoit.com\r\n",
    "\r\nSenior Applications Engineer\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8001\r\nMobile: 408.394.3613\r\nFax: 408.562.8100\r\ndfox@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nYour Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.5622-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "\r\n---\r\nRicardo Garcia\r\nTechnical Marketing Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\n(408) 829-4827 cell\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAmy Moore\r\nExecutive Assistant\r\nMark Tapling, President & CEO\r\nMatt DiMaria, Sr. VP Marketing & Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8106  fax\r\n(408) 857-8520  mobile\r\nwww.avocadoit.com <http://www.avocadoit.com>\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nDavid Marcus\r\nSenior Corporate Counsel\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nBrett W. Welborn\r\nAccount Development\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nKhou Fang\r\nTechnical Recruiting Manager\r\nkfang@avocadoit.com\r\n408-562-8096\r\n",
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225\r\nJohn.McCann@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!\r\n--------------------------\r\nSent from my Wireless Handheld',
    "\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n2211 North First Street, \r\nSuite 200, San Jose,\r\nCA  95131\r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRon\r\nAvocadoIT, Your Business Everywhere! \r\n********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nwork (408) 562-8039 \r\nmobile (415) 845-6789 \r\nfax (408) 562-8100 \r\nwww.avocadoit.com \r\n**********************************************",
    "\r\nEdward Tian\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8122\r\nFax: 408.562.8100 \r\netian@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRon\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************",
    "Toshiya Otani\r\nVice President, Business Development - Asia Pacific\r\nAvocadoIT, Inc.\r\nRepresentative Director\r\nAvocadoIT Japan KK\r\ntel: 650-740-9792 (cell/US), +81-90-6019-0079 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nwww.avocadoit.com\r\n",
    "\r\nTy\r\n(415) 336-8668 Mobile\r\nAvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nT. Ty Wang\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7945 Office\r\n(408) 562-8100 Fax\r\nty.wang@avocadoit.com\r\n*********************************************",
    "\r\n--\r\nAvocadoIT Europe GmbH\r\nHamborner Strasse 53\r\n40472 Duesseldorf\r\nphone\t+49 (211) 94098-119\r\nfax\t+49 (211) 94098-200\r\nmobile\t+49 (171) 4014644\r\nmailwww.avocadoit.com",
    "\r\nRon (www.avocadoit.com)",
    "\r\n--------------------------\r\nRicardo Garcia                         rgarcia@avocadoit.com\r\nSales Engineer                         408-829-4827 m\r\nAvocadoIT Inc.                          408-562-7984 w\r\n---\r\nSent from my AvocadoIT mobile device (www.avocadoit.com)",
    "\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\nKhou Fang\r\nTechnical Recruiting Manager\r\nkfang@avocadoit.com\r\n408-562-8096",
    "\r\n---\r\nD'Arcy Salzmann, Product Manager\r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc.\r\n49 Ontario Street, 7th floor, Box 24\r\nToronto, Ontario, M5V 2V1\r\nCanada \r\nphone    +1.416.643.3593 Toronto, Ont.\r\nphone    +1.408.562.8172 San Jose, CA. \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook address book\r\n< http://my.infotriever.com/darcysalzmann>",
    "\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************",
    "\r\nMiyuki\r\nMiyuki Goldman\r\nQA Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7979\r\nFax: 408.562.8100\r\nmiyuki.goldman@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\n----------------------------------------------------------------------------------------\r\nAmit Sethi                                            asethi@avocadoit.com\r\nSr. Director, Business Development        408.562.7966 Office\r\nAvocadoIT, Inc.                                      408.562.8100 Fax\r\n3231 Scott Blvd.                                    www.avocadoit.com\r\nSanta Clara, CA 95054",
    "\r\nBrett Welborn\r\nAccount Development Manager\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com",
    "\r\nKelsey Kerr\r\nDirector Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408.562.8036 (office)\r\n408.394.1054 (cell)\r\n408.562.8100 (fax)\r\nkkerr@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\ndbm",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 650.766.6094\r\nFax: 408.562.8100\r\ndmargulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057\r\nAvocadoIT                                 www.avocadoit.com",
    "\r\nRuth Gantly\r\nManager, Information Products\r\nAvocadoIT\r\n3231 Scott Blvd.\r\nSanta Clara, CA 95054\r\nemail rgantly@avocadoit.com\r\nPhone: 408.562.8033\r\nFax: 408.562.8100",
    "\r\nBest regards,\r\n-------------------------------------------------------------------------------------------------------------\r\nRichard Evans\r\nDeveloper Support Engineer\r\nL&H Speech and Language Products\r\nrevans@lhsl.com\r\nsupport@lhsl.com",
    "\r\n____________________________________\r\nDiana Y. Chen\r\nApplications Engineer\r\nOffice: (408) 562-8143\r\nCell:  (408) 829-4689\r\nDiana.Chen@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)',
    '\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)',
    '\r\nBrett W. Welborn\r\nAccount Development\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)',
    "\r\nBrett Welborn\r\nAccount Development Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95134\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95134\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nBrett W. Welborn \r\nAccount Development \r\nAvocadoIT, Inc. \r\nhttp://www.avocadoit.com \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\n(800) 355-1103 ext. 8186 \r\n(408) 562-8186 (direct)",
    "\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 562-8186",
    "Brett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nBrett W. Welborn\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\n> 3231 Scott Boulevard\r\n> Santa Clara, CA  95054\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "\r\nElisabeth V. Whaley\r\nAssist. to the VP of Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAssit. to the VP of Client Srvcs.\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Sales\r\n3231 Scott Blvd.\r\nSanta Clara, CA  95054\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAssit. to the VP of Customer Service\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "Elisabeth V. Whaley\r\nAssit. to the VP of Customer Service\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95113\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95113\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. in Sales\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Assist. Prof. Serv./Sales\r\n2211 North First Street\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAdmin. Assist. in Sales\r\n2211 North First Street\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAdmin. Assist. in Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8100\r\nElisabeth.Whaley@avocadoit.com",
    "Elisabeth V. Whaley\r\nAssist. to the VP of Prof. Srvcs.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com",
    "\r\nElisabeth V. Whaley\r\nAdmin. Asst. to VP of Cust. Serv.\r\n2211 North First St., Ste. 200\r\nSan Jose, CA  95131\r\nphone: 408-562-8189\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nElisabeth V. Whaley\r\nExec. Assit. to the VP of Client Srvcs.\r\noffice: (408) 562-8189\r\nfax: (408) 562-8103\r\nElisabeth.Whaley@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nNoemi Petrone \r\nExecutive Administrator to Venk Shukla \r\nAvocadoIT \r\n3231 Scott Blvd. \r\nSanta Clara, CA \r\n408-562-8032 (direct) \r\n408-588-1519 (fax)",
    "Noemi Petrone\r\nExecutive Administrator to Venk Shukla\r\nAvocadoIT\r\n3231 Scott Blvd.\r\nSanta Clara, CA\r\n408-562-8032 (direct)\r\n408-588-1519 (fax)",
    "\r\nNoemi Petrone\r\nExecutive Administrator to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd.\r\nSanta Clara, CA\r\n408-562-8032 (direct)\r\n408-588-1519 (fax)",
    "\r\nNoemi Petrone\r\n2179 Coolidge Dr.\r\nSanta Clara, CA\r\n408-246-8811\r\n",
    "\r\nTrish Lady\r\nVP, Business Development - Content\r\nAvocadoIT\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  408-562-8070   f:  408-562-8100   m:  415-505-3756\r\nAre you ready for the wireless web?",
    "\r\nRegards,\r\nTrish",
    '\r\nTrish',
    "r\nhttp://concepts.bigcharts.com/marketwatch/mktw-wireless-browser/index.html \r\nJoe Techar\r\nDirector\r\nCBS MarketWatch.com\r\n612.752.1138\r\njtechar@marketwatch.com\r\n123 North 3rd St., Suite 300\r\nMinneapolis, MN 55401\r\n",
    "\r\nMike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com\r\n*** AvocadoIT, Your Business Everywhere! ***",
    '\r\nMike\r\n--------------------------\r\nMike Richman\r\nSenior Account Executive\r\n415-460-0644 W\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nMike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com",
    'Mike\r\n--------------------------\r\nMike Richman\r\nSenior Account Executive\r\n415-460-0644 W\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nTamra Van Steenburgh\r\nDirector of Strategic Finance\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nMain:  408-562-8000\r\nDirect:  408-562-8087\r\nFax:  408-562-8100\r\ntamra.vansteenburgh@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nMike Richman\r\nSenior Account Manager\r\nAvocadoIT, Inc.\r\nDirect: 415.460.0644 \r\nMike.Richman@AvocadoIT.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n--> Andy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8102 fax\r\nwww.avocadoit.com\t\t\t\t2211 N. 1st Street, Suite 200, San Jose, CA 95131",
    "\r\n--> Andy\r\n_____________\r\nAndrew Wong\r\nAvocadoIT, Your Business Everywhere!\r\n408.562.8131\r\nandy.wong@avocadoit.com",
    "\r\nAndy\r\n---\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nThanks!\r\n--> Andy\r\n_____________\r\nAndrew Wong\r\nAvocadoIT - Your Business Everywhere\r\n408.562.8131\r\nandy.wong@avocadoit.com",
    "\r\nandy\r\n---\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nThanks!\r\n--> Andy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nThanks!\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa\r\nClara, CA 95054",
    "\r\n--> Andy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector Solutions Marketing\r\n408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8102 fax\r\nwww.avocadoit.com\t\t\t\t2211 N. 1st Street, Suite 200, San Jose, CA 95131\r\nYour Business Everywhere",
    "\r\nthanks!\r\nandy\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    '\r\nPeter Smialek\r\nDirector, North American Sales\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek\r\nRegional Sales Director\r\nAvocadoIT\r\n781 238 6774',
    '\r\nPeter Smialek\r\nSales Director\r\nAvocadoIT, Inc.\r\n25 Mall Road - Suite 300\r\nBurlington, MA 01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek \r\nRegional Sales Director \r\nAvocadoIT \r\n781 238 6774',
    'Peter Smialek\r\nSales Director\r\nAvocadoIT, Inc.\r\n25 Mall Road - Suite 300\r\nBurlington, MA 01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek\r\nDirector, North American Sales\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile',
    '\r\nThanks\r\nNadir Nawabzada\r\nSystem Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8044\r\nCell: 408-205-7289',
    '\r\nPeter Smialek\r\nDirector\r\nAvocadoIT, Inc.\r\n25 Mall Road - Suite 300\r\nBurlington, MA 01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    "\r\n--------------------------\r\nThe message is brief because it is sent from my Blackberry\r\nVenktesh Shukla\r\nAvocadoIT : Your business everywhere",
    '\r\nPeter Smialek \r\nDirector, North American Sales \r\nAvocadoIT \r\n781 238 6774  Office \r\n781 229 0330  Fax \r\n617 901 9111  Mobile \r\n"AvocadoIT...Your Business Everywhere"',
    "\r\n[This message was sent from Blackberry handheld.]",
    "\r\nhttp://iwww.sitraka.com/mobility/rd/aa/index.html\r\nkgr\r\n--\r\nKeith Ritchie\r\nTeam Lead\r\nMobile Infrastructure\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    '\r\nLisa Zissen \r\nExecutive Administrator\r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite #200\r\nSan Jose, California  95131 \r\n(408) 562-8012 - direct \r\n(408) 205-0665 - Mobile \r\n(408) 562-8100 - Fax \r\nlisa.zissen@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, your business everywhere! <?xml:namespace prefix = o ns = "urn:schemas-microsoft-com:office:office" />',
    "\r\nhttp://iwww.sitraka.com/mobility/rd/aa/may15-21.html\r\nkgr\r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
     "\r\nEnjoy,\r\nkgr\r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "Ritchie\r\nTeam Lead\r\nMobile Infrastructure\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "\r\n---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    'r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n',
    "\r\n--\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "\r\nkgr",
    "\r\nKeith\r\n__________________________________________________\r\nKeith Ritchie\r\nDevelopment Manager\r\nAvocadoIT Canada\r\n49 Ontario Street\r\nBox 24, 6th Floor\r\nToronto ON M5A 2V1\r\nPhone: 416.366.6425 x 306\r\nFax: 416.643.4832\r\nkeith.ritchie@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n__________________________________________________\r\nKeith Ritchie\r\nDevelopment Manager\r\nAvocadoIT Canada\r\n49 Ontario Street\r\nBox 24, 6th Floor\r\nToronto ON M5A 2V1\r\nPhone: 416.366.6425 x 306\r\nFax: 416.643.4832\r\nkeith.ritchie@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nthx,\r\nKeith Ritchie\r\nDevelopment Manager, Respond Platform\r\nSitraka Mobility\r\nhttp://www.sitraka.com/mobility\r\nWork  - (416) 366-6425 x 306\r\nCell  - (905) 467-1181\r\nFax  - (416) 214-1235\r\nEmail  - keith.ritchie@sitraka.com\r\nBlackberry - sitraka2@mobile.rogers.com\r\n",
    "\r\n--------------------------\r\nGrant Frederiksen\r\nAvocadoIT, Inc.\r\nSales Director - SE\r\n770.753.9837\r\ngrantf@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n--------------------------\r\nGrant Frederiksen\r\nAvocadoIT, Inc.\r\nSales Director - SE\r\n770.753.9837\r\ngrantf@avocadoit.com\r\nSent from my AvocadoIT Powered Wireless Handheld (www.BlackBerry.net)",
    "\r\nGrant\r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nGrant  \r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nGrant \r\nAre you ready for the wireless web?\r\n",
    "\r\n--------------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n--------------------------\r\nSent from my BlackBerry Wireless Handheld",
    "\r\n--------------------------\r\nLarry Salerno\r\nAvocadoIT, Inc\r\nSent from my BlackBerry Wireless Handheld",
    "\r\nLisa Zissen\r\nExecutive Assistant\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "--------------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n--------------------------\r\nR. Scott Isgitt\r\nSales Director\r\nAvocadoIT\r\n512-514-6424 - Office\r\n512-636-1275\r\nscott.isgitt@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)"
    "\r\n--------------------------\r\nStephanie Moss\r\nSent from my BlackBerry Wireless Handheld\r\n",
    "\r\nLisa Zissen\r\nExecutive Assistant\r\nAvocadoIT, Inc.\r\nSent wirelessly from my Blackberry handheld device.\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\n-------------------------- \r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net) \r\n",
    "\r\n--------------------------\r\nR. Scott Isgitt\r\nSales Director\r\nAvocadoIT\r\n512-514-6424 - Office\r\n512-636-1275\r\nscott.isgitt@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)"
    "\r\nJosh\r\n--------------------------------\r\nSent from my BlackBerry Wireless Handheld\r\n(www.BlackBerry.net)\r\n",
    "--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)\r\n",
    "--------------------------\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)",
    "\r\nRoberta Wiggins, Director Wireless Mobile\r\nYankee Group\r\nGloria\r\n____________________________\r\nGloria Chon\r\nAccount Associate\r\nCandelori Communications\r\n1270 Oakmead Parkway, Suite 314\r\nSunnyvale, CA  94086\r\nTel   408.774.3414 x227\r\nFax  408.774.3419\r\ngchon@candelori.com\r\n",
    #"\r\nEnabling Infrastructure for Post-PC Era\r\n",
    'AvocadoIT, Your Business Everywhere! \r\n********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nwork (408) 562-8039 \r\nmobile (415) 845-6789 \r\nfax (408) 562-8100 \r\nwww.avocadoit.com \r\n**********************************************',
    '\r\nEnabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\n3231 Scott Boulevard \r\nSanta Clara, CA  95054 \r\nTel: (408) 562-8039 \r\nMobile: (415) 845-6789 \r\nFax: (408) 408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************',
    '\r\nAvocadoIT, Your Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************',
    "\r\nEnabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8039 \r\nMobile: 415.845.6789 \r\nFax:  408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************",
    "\r\nRon\r\nAvocadoIT, Your Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "\r\nEnabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100 \r\nrsilverton@avocadoit.com\r\n************************************",
    "\r\nAvocadoIT, Your Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector, AvocadoITOnline\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************",
    "Your Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.5622-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "Ron Enabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\n3231 Scott Boulevard \r\nSanta Clara, CA  95054 \r\nTel: (408) 562-8039 \r\nMobile: (415) 845-6789 \r\nFax: (408) 408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************ \r\n",
    "\r\nYour Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "Enabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8039 \r\nMobile: 415.845.6789 \r\nFax:  408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************",
    "\r\nYour Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.5622-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    "AvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************\r\n<<Ron Silverton.vcf>>\r\n",
    "Enabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8039\r\nMobile: (415) 845-6789\r\nFax: (408) 408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************\r\n",
    "Enabling Infrastructure for Post-PC Era \r\n************************************ \r\nRon D Silverton \r\nDirector of Strategic Marketing \r\n3231 Scott Boulevard \r\nSanta Clara, CA  95054 \r\nTel: (408) 562-8039 \r\nMobile: (415) 845-6789 \r\nFax: (408) 408.562.8100 \r\nrsilverton@avocadoit.com \r\n************************************",
    "\r\nRon\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "\r\n*********************************\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************",
    "\r\nThanks,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nMobilized by AvocadoIT Inc.",
    '\r\nThanks,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n***********************************************\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nThanks, \r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nBrought to you wirelessly by AvocadoIT Inc.',
    '\r\nThanks guys,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nMobilized by AvocadoIT Inc.',
    '\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n***********************************************\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\n--------------------------\r\nR. Scott Isgitt\r\nSales Director\r\nAvocadoIT\r\n512-514-6424 - Office\r\n512-636-1275\r\nscott.isgitt@avocadoit.com\r\nSent from my BlackBerry Wireless Handheld (www.BlackBerry.net)',
    '\r\nDon\r\n--------------------------\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nScott\r\n--------------------------\r\nAvocadoIT - Your Business Everywhere!\r\nScott Weller\r\n206-323-5105 (o)\r\n206-604-6105 (m)',
    '\r\nThanks!\r\nAndy\r\n--------------------------\r\nSent to you wirelessly from AvocadoIT - Your Business Everywhere\r\nAndy Wong\r\n408.562.8131\r\nawong@avocadoit.com',
    '\r\nAmit.\r\n-------------------------------\r\nAvocadoIT - Your Business Everywhere!',
    '\r\n--------------------------\r\n"AvocadoIT...Your Business Everywhere"',
    '--------------------------\r\n"AvocadoIT...Your Business Everywhere"',
    '--------------------------\r\nAvocadoIT , Your Business Everywhere!',
    '\r\n-------------------------------\r\nAvocadoIT - Your Business Everywhere!',
    '-------------------------------\r\nAvocadoIT - Your Business Everywhere!',
    '\r\nAvocadoIT - Your Business Everywhere!\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!',
    '\r\nJanie\r\n(303) 679-9779\r\nAvocadoIT - Your Business Everywhere!',
    '\r\njanie\r\nAvocadoIT - Your Business Everywhere!',
    '\r\nAvocadoIT - Your Business Everywhere!',
    '\r\nPeter Smialek\r\nAvocadoIT\r\n25 Mall Road - Suite 300\r\nBurlington,  MA  01803\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\nPeter Smialek \r\nSales Director \r\nAvocadoIT, Inc. \r\n25 Mall Road - Suite 300 \r\nBurlington, MA 01803 \r\n781 238 6774  Office \r\n781 229 0330  Fax \r\n617 901 9111  Mobile \r\n"AvocadoIT...Your Business Everywhere"',
    '\r\n--> Andy\r\n_____________\r\nAndrew Wong\r\nAvocadoIT - Your Business Everywhere\r\n408.562.8131\r\nandy.wong@avocadoit.com',
    '\r\n------------------------------------\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n--------------------------\r\nDoug Adkins\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nGermana Martinez\r\nHuman Resources Coordinator,\r\nand Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nChris\r\n***************************\r\nChristopher Quinter\r\nDirector of Sales\r\n(267) 757-0736  Main\r\n(215) 778-3235  Cell\r\n(413) 714-7699  Fax\r\n***************************\r\nAvocadoIT, Your Business Everywhere!',
    "\r\n--------------------------\r\nRicardo Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7984 (direct)\r\n(408) 829-4827 (mobile)\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nChris\r\n*******************************************************************\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Mail\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com \r\n*******************************************************************",
    "\r\nAvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nRobert Moore\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com\r\n*********************************************",
    '\r\nAvocadoIT, Your Business Everywhere!\r\n--\r\nAvocadoIT Europe GmbH\r\nKönigsallee 106\r\n40215 Düsseldorf, Germany\r\nphone\t+49 (0) 211.30122-125\r\nfax \t+49 (0) 211.30122-200\r\ncellular\t+49 (0) 171.3845255\r\nmailwww.avocadoit.de',
    '\r\n-Howard\r\n____________________________\r\nHoward Mora\r\nQA Director\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7954 (direct)\r\n(408) 562-8000 (main)\r\n(408) 562-8100 (fax)\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225\r\nJohn.McCann@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!\r\n---------------------',
    '\r\nMatt\r\nAvocadoIT, Your Business Everywhere!\r\n**********************************************\r\nMatt DiMaria\r\nSVP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    "\r\n--------------------------\r\nA Wireless Message from AvocadoIT\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-857-8729\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nThanks!\r\nAndy\r\n--------------------------\r\nSent to you wirelessly from AvocadoIT, Your Business Everywhere!\r\nAndy Wong\r\n408.562.8131\r\nawong@avocadoit.com',
    "\r\nGermana Martinez\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax\r\nAvocadoIT, Your Business Everywhere!",
    '\r\n"AvocadoIT, Your Business Everywhere!" \r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp',
    '\r\nEnjoy,\r\nJimmy\r\n--------------------------------------\r\nJimmy Kim\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n4933 West Louise\r\nSuite 101\r\nSkokie, IL 60077\r\nOffice: 312-409-4919\r\nMobile: 312-405-3836\r\nFax: 810.279.4299\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    "\r\nThanks,\r\nTom\r\nThomas J. Stoddard \r\nDistrict Manager, West \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n949 425-9826 Direct\r\ntom.stoddard@avocadoit.com \r\nAvocadoIT, Your Business Everywhere!",
    '\r\nThanks\r\nRavi  Dadi\r\nAvocadoIT, Inc\r\n408-562-8138\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nMain: 408.562.8000 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, Your Business Everywhere!'
    '\r\nSincerely,\r\nJohn Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail\r\nAvocadoIT, Your Business Everywhere!',
    "--------------------------\r\nAvocadoIT, Your Business Everywhere! (www.avocadoit.com)",
    '\r\n____________________________________\r\nDiana Y. Chen\r\nApplications Engineer\r\nOffice: (408) 562-8143\r\nDiana.Chen@avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    '\r\n***************************\r\nChristopher Quinter\r\nDirector of Sales\r\n(267) 757-0736  Main\r\n(215) 778-3235  Cell\r\n(413) 714-7699  Fax\r\n***************************\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nThank you and have a great day! \r\nChris \r\n******************************************************************* \r\nChristopher Quinter \r\nDirector of Sales \r\n431 Trumbull Court \r\nNewtown, PA  18940 \r\n(267) 757-0736  Mail \r\n(215) 778-3235  Mobile \r\n(413) 714-7699  Fax \r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com \r\n*******************************************************************',
    '\r\nThank you and have a great day!\r\nChris\r\nChristopher Quinter\r\nAccount Manager\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Office\r\n(215) 778-3235  Mobile\r\nAvocadoIT, Your Business Everywhere! www.avocadoit.com\r\n',
    'dbm\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com \r\nAvocadoIT, Your Business Everywhere!',
    '\r\nSincerely,\r\nJohn Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail\r\nAvocadoIT, Your Business Everywhere!',
    "\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nMain: 408.562.8000 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, Your Business Everywhere!",
    "\r\nRegards,\r\nAvocadoIT, Your Business Everywhere!\r\nJeff Newman",
    "\r\nOtto Mok\r\nTechnical Team Lead\r\nAvocadoIT Canada\r\nMain: 416.366.6425 x865\r\nDirect: 416.643.4865\r\nMobile: 416.346.3834\r\nFax: 416.643.4832\r\nEmail: otto.mok@avocadoit.com\r\nWebsite: http://www.avocadoit.com/\r\nAvocadoIT, Your Business Everywhere!",
    "\r\nThank you and have a great day!\r\nChris\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Main\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax\r\nAvocadoIT, Your Business Everywhere!  www.avocadoit.com",
    "\r\nTy\r\nAvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nT. Ty Wang\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7945 Office\r\n(408) 562-8100 Fax\r\nty.wang@avocadoit.com\r\n*********************************************",
    "\r\nTy\r\nT. Ty Wang\r\nAvocadoIT, Inc.\r\nBusiness Development\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7945 Office   (408) 562-8100 Fax\r\nty.wang@avocadoit.com",
    "\r\nTy\r\nT. Ty Wang\r\nDirector, Business Development\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7945\r\nFax: 408.562.8100\r\nty.wang@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    "\r\n-------------------\r\nAvocadoIT, Your Business Everywhere!",
    '\r\n--------------------------\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nThanks,\r\nTom\r\ntom.stoddard@avocadoit.com\r\n949 425-9826  Office\r\n949 300-3372  Mobile\r\nAvocadoIT, Your Business Everywhere!\r\nwww.avocadoit.com',
    "\r\nThanks!\r\n--> Andy\r\n_____________\r\nAndy Wong\r\nDirector, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8131\r\nFax: 408.562.8131\r\nawong@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nRay\r\n------------------------------\r\nRay Canuel \r\nSales Engineer          \r\n703.281.4223  W\r\n703.728.2007  M\r\n"AvocadoIT, Your Business Everywhere!"',
    "Germana Martinez\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax\r\nAvocadoIT, Your Business Everywhere!",
    '\r\nRajeev\r\n--------------------------------------------\r\nRajeev Mohindra\r\nFounder & V.P. of Engineering\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8002\r\nFax: 408.562.8100\r\nrajeev@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, Your Business Everywhere!',
    'Your Business Everywhere\r\n-- \r\nAvocadoIT Europe GmbH \r\nKönigsallee 106 \r\n40215 Düsseldorf, Germany \r\n(p)     +49 (0) 171.3845255 \r\n(f)     +49 (0) 171.13.3845255\r\nmailwww.avocadoit.com',
    '\r\nAvocadoIT, Your Business Everywhere!',
    '\r\nThanks,\r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nKamiyacho Mori Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nKamiyacho Mori Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp\r\n"AvocadoIT, Your Business Everywhere!"',
    '-------------------------------------------------------------------\r\n** AvocadoIT, Your Business Everywhere **',
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc.\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice 773-243-0324\r\nFax  773-243-0225\r\nwww.AvocadoIT.com\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRegards,\r\nSanjay Dubey\r\nAvocadoIT Inc. R&D\r\nsanjay.dubey@avocadoit.com\r\n(408) 562 7975\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRavi  Dadi\r\nAvocadoIT, Inc\r\n408-562-8138\r\n"AvocadoIT, Your Business Everywhere!"',
    'AvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nRobert Moore\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com\r\n*********************************************\r\n',
    '--------------------------\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRay Canuel \r\nSales Engineer          \r\n703.281.4223  W\r\n703.728.2007  M\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nJohn McCann, District Manager\r\nAvocadoIT, Inc.\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice 773-243-0324\r\nFax  773-243-0225\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nMark Stewart\r\nAvocadoIT, Inc.\r\nYour Business Everywhere",
    '\r\nRoopak\r\nRoopak Parikh \r\nAvocadoIT Inc. R&D \r\nroopak@avocadoit.com \r\noff: (408) 562 8023 \r\ncell: (408 )857 8531 \r\n“AvocadoIT, Your Business Everywhere!”',
    'Mike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com\r\n*** AvocadoIT, Your Business Everywhere! ***',
    'r\nThanks.\r\nAlex\r\n--------------------------\r\nAlex Tran\r\nDirector of Product Marketing\r\n408.562.8068 Work\r\n408.420.3877 Mobile\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nDiana\r\n____________________________________\r\nDiana Y. Chen\r\nApplications Engineer\r\nOffice: (408) 562-8143\r\nDiana.Chen@avocadoit.com\r\nAvocadoIT, Your Business Everywhere.',
    '\r\nThanks.\r\nAlex\r\n--------------------------\r\nAlex Tran\r\nDirector of Product Marketing\r\n408.562.8068 Work\r\n408.420.3877 Mobile\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\n******************************************************************* \r\nChristopher Quinter \r\nDirector of Sales \r\n431 Trumbull Court \r\nNewtown, PA  18940 \r\n(267) 757-0736  Mail \r\n(215) 778-3235  Mobile \r\n(413) 714-7699  Fax   www.avocadoit.com \r\n*******************************************************************',
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Your Business Everywhere!",
    "AvocadoIT, Your Business Everywhere!\r\n*********************************************\r\nRobert Moore\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com\r\n*********************************************",
    "\r\nChristopher Quinter\r\nDirector of Sales\r\n(267) 757-0726  Office\r\n(215) 778-3235  Cell\r\nAvocadoIT, Your Business Everywhere",
    '\r\nRoopak\r\nRoopak Parikh\r\nAvocadoIT Inc. R&D\r\nroopak@avocadoit.com\r\noff: (408) 562 8023\r\ncell: (408 )857 8531\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\n"AvocadoIT, Your Business Everywhere!"\r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp',
    '\r\n*** AvocadoIT, Your Business Everywhere! ***\r\n',
    'AvocadoIT, Your Business Everywhere!\r\n',
    'AvocadoIT, Your Business Everywhere! \r\n********************************************** \r\nMatt DiMaria \r\nSVP, Marketing and Business Development \r\nAvocadoIT, Inc. \r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nwork (408) 562-8040 \r\nwww.avocadoit.com \r\n**********************************************',
    '--------------------------\r\n"AvocadoIT... Your Business Everywhere"',
    '**********************************************\r\nMatt DiMaria\r\nSVP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '**********************************************\r\nMatt DiMaria\r\nSVP, Marketing and Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8040\r\nwww.avocadoit.com\r\n**********************************************',
    '**********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nwork (408) 562-8039\r\nmobile (415) 845-6789\r\nfax (408) 562-8100\r\nwww.avocadoit.com\r\n**********************************************',
    'Peter Smialek\r\nDirector, North American Sales\r\nAvocadoIT\r\n781 238 6774  Office\r\n781 229 0330  Fax\r\n617 901 9111  Mobile\r\n"AvocadoIT...Your Business Everywhere"',
    '\r\njm\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\nYour Business Everywhere!\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 ~ phone\r\n408.891.8144 ~ cell\r\nwww.avocadoit.com',
    '\r\nThanks!\r\n"AvocadoIT, Your Business Everywhere!" \r\nNaruhito Morooka\r\nAvocadoIT Japan K.K.\r\nTEL: 03-5404-3420\r\nFAX: 03-5404-3401\r\nURL: http://www.avocadoit.com\r\nMail',
    '\r\nRoopak Parikh\r\nAvocadoIT Inc. R&D\r\nroopak@avocadoit.com\r\noff: (408) 562 8023\r\ncell: (408 )857 8531\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nNobu\r\nNobuyuki Takaki\r\nSenior Director\r\nBusiness Development & Marketing\r\nAvocadoIT Japan K.K.\r\nKamiyacho MT Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: http://www.avocadoit.co.jp/\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nNobuyuki Takaki\r\nSenior Director\r\nBusiness Development & Marketing\r\nAvocadoIT Japan K.K.\r\nKamiyacho MT Bldg. 14th Floor\r\n4-3-20 Toranomon Minato-Ku\r\nTokyo 105-0001 Japan\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: http://www.avocadoit.co.jp/\r\n"AvocadoIT, Your Business Everywhere!"',
    '"AvocadoIT, Your Business Everywhere!" \r\nNobuyuki Takaki\r\nBusiness Development Director\r\nAvocadoIT Japan K.K.\r\nMail:nobuyuki.takaki@avocadoit.com\r\nPhone:81-3-5404-3418\r\nHP: www.avocadoit.com/jp',
    '\r\nLarry\r\nLarry Salerno\r\nAvocadoIT, Inc\r\nOffice: 847-277-0600\r\nFax:    847-277-0640\r\nMobile:847-922-1006\r\n"AvocadoIT, Your Business Everywhere!"',
    'Your Business Everywhere\r\n-- \r\nAvocadoIT Europe GmbH \r\nKönigsallee 106 \r\n40215 Düsseldorf, Germany \r\n(p)     +49 (0) 211.30122-125 \r\n(f)     +49 (0) 211.30122-200 \r\n(m)     +49 (0) 171.3845255 \r\nmailwww.avocadoit.com',
    '\r\nRegards,\r\nAlex\r\n--------------------------\r\nAlex Tran\r\nDirector of Product Marketing\r\n408.562.8068 Work\r\n408.420.3877 Mobile\r\n"AvocadoIT, Your Business Everywhere!"',
    '\r\nRegards,\r\nAvocadoIT, Your Business Everywhere\r\n============================================\r\nDavid M. Hosch,  Sales Manager\r\nAvocadoIT Inc.\r\n49 Ontario Street, 7th Floor\r\nToronto, Ontario,Canada, M5A  2V1\r\nTel: 416-643-4829, Cell: 416-460-3557\r\nFax:416-214-1235\r\nEmail: david.hosch@avocadoit.com\r\nBlackberry: david.hosch@mobile.rogers.com\r\n',
    '\r\n"AvocadoIT, Your Business Everywhere!" \r\nNaruhito Morooka\r\nDirector of Sales & Marketing\r\nAvocadoIT Japan K.K.\r\nTEL: 03-5404-3420\r\nFAX: 03-5404-3401\r\nURL: http://www.avocadoit.com\r\nMail',
    '\r\nGermana Martinez\r\nHuman Resources Coordinator, \r\nSales Executive Admin. and\r\nExecutive Assistant to the CFO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX\r\nAvocadoIT, Your Business Everywhere',
    '\r\nGermana Martinez\r\nHuman Resources Coordinator, \r\nSales Executive Admin. and\r\nExecutive Assistant to the CFO\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX\r\nAvocadoIT, Your Business Everywhere',
    '\r\nGermana Martinez\r\nHuman Resources\r\nAvocadoIT, Your Business Everywhere',
    "\r\nDavid Swanson\r\nHuman Resources\r\nAvocadoIT, Your Business Everywhere",
    "\r\nRegards,\r\nAlex Tran\r\nDirector of Product Marketing\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 562-8068 office\r\n(408) 562-8100 fax\r\n(408) 420-3877 cell\r\natran@avocadoit.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n650.270.4309\r\nwww.avocadoit.com",
    "\r\nLisa Zissen\r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\n-Lisa\r\nLisa Zissen\r\nExec. Asst. to Prakash Iyer, CTO & Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen\r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite #200\r\nSan Jose, CA  95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8012\r\nFax: 408.562.8104\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen \r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 205-0665 - Mobile\r\n(408) 562-8100 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa\r\nLisa Zissen \r\nExecutive Assistant to Prakash Iyer \r\nFounder, CTO and Sr. VP of Engineering Operations \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\n2nd Floor \r\nSan Jose, California  95131 \r\n(408) 562-8012 - direct \r\n(408) 205-0665 - Mobile \r\n(408) 562-8100 - Fax \r\nlisa.zissen@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, your business everywhere! \r\n_____  \r\nUpgrade Your Email - Click here!",
    "\r\nThanks.\r\nStuart\r\nStuart Finn\r\nSenior Vice President,\r\nSales and Service\r\nAVOCADOIT, INC.\r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-7950 (Office) \r\nstuart.finn@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nTamra Van Steenburgh\r\nFinance Department\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd.\r\nSanta Clara, CA  95054\r\nMain:  408-562-8000\r\nDirect:  408-562-8087\r\nFax:  408-562-8100\r\ntvansteenburgh@avocadoit.com",
    "\r\nSusan Simmons\r\nDirector of Finance & Administration\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n(408) 562-8022 direct dial\r\n(408) 562-8000 corporate\r\n(408) 562-8100 fax\r\nssimmons@avocadoit.com",
    "\r\nKelsey Kerr\r\nDirector Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408.562.8036 (office)\r\n408.394.1054 (cell)\r\n408.562.8100 (fax)\r\nkkerr@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndmargulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAndrea Peterson\r\nSales Administrative Assistant\r\n3231 Scott Blvd. \r\nSanta Clara, CA  95054\r\np(408) 562-8056\r\nf(408) 562-8100\r\napeterson@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRegards,\r\nPhilip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\nRyan Glover\r\nPublic Relations Coordinator \r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408.562.8057 office\r\n408.420.3841 mobile\r\nrglover@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nDavid Margulies\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8119\r\nDavid.Margulies@avocadoit.com\r\nthanks,\r\nRobert Moore",
    "\r\n--- \r\nD'Arcy Salzmann, Product Manager \r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc. \r\n260 King Street East, 3rd Floor \r\nToronto, ON, Canada M5A 4L5 \r\n+1.416.643.3593 phone / +1.416.605.1076 fax \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook address book \r\nhttp://my.infotriever.com/darcysalzmann \r\n-----",
    "\r\n--- \r\nD'Arcy Salzmann, Product Manager \r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc. \r\n260 King Street East, 3rd Floor \r\nToronto, ON, Canada M5A 4L5 \r\n+1.416.643.3593 phone / +1.416.605.1076 fax \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook address book \r\nhttp://my.infotriever.com/darcysalzmann",
    "\r\n--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com",
    "\r\n----------------------------------------------------------------------------------------\r\nAmit Sethi                                            asethi@avocadoit.com\r\nVice-President, Business Development   408.562.7966 Office\r\nAvocadoIT, Inc.                                      408.562.8100 Fax\r\n2211 N. First Street, St. 200                   www.avocadoit.com\r\nSan Jose, CA 95131",
    "\r\nRepresentative Director\r\nAvocadoIT Japan KK\r\ntel: 650-740-9792 (cell/US), +81-90-6019-0079 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nwww.avocadoit.com",
    "\r\nKind Regards,\r\nLaura\r\nLaura C. Henderson\r\nClient Development\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nhttp://www.avocadoit.com",
    "Lisa Quaglietti\r\nVP Client Development & Services\r\nAvocadoIT\r\n3231 Scott Blvd\r\nSanta Clara, CA 95054\r\nlisaq@avocadoit.com\r\n408-562-8120 - Direct\r\n408-420-3860 - Cell",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8059\r\nMobile: (650) 766-6094\r\nFax: (408) 588-0060\r\ndmargulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAndrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n3231 Scott Blvd\r\nSanta Clara, CA 95054\r\nandrew.wagoner@avocadoit.com",
    "\r\nSusan Simmons\r\nDirector of Finance & Administration\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nDirect dial: (408) 562-8022\r\nCorporate phone: (408) 562-8000\r\nCorporate fax: (408) 562-8100\r\nsusan.simmons@avocadoit.com",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Inc. ",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, the leader in mobile wireless platforms for the Corporate market\r\n-----------------------------",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\nSent wirelessly from AvocadoIT - The leading Enterprise Mobility Company\r\nDavid C. Meltzer, Esq.  Director of Sales AvocadoIT",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT - The leading Enterprise Mobility Company Philip O'Connell, Esq.  Director of Sales AvocadoIT",
    "\r\n-------------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\nThanks,\r\nTy\r\n--------------------------\r\nSent wirelessly by AvocadoIT from my BlackBerry Handheld",
    "--------------------------\r\nSent wirelessly from AvocadoIT, Inc. ",
    "\r\nElba\r\nSent wirelessly from AvocadoIT, Inc.",
    "\r\n----------------------------------\r\nJohn Schemena                \r\nDirector Applications Engineering          \r\n408 562-8028  W\r\n408 829-8236  M\r\nwww.avocadoit.com \r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT.",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, the leader in mobile wireless platforms for the Corporate market\r\n------------------------",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT, Inc.",
    "\r\nAmit.\r\n-------------------------------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\nTy\r\n--------------------------\r\nSent wirelessly by AvocadoIT from my BlackBerry Handheld",
    "\r\n-------------------\r\nSent wirelessly from AvocadoIT, Inc.",
    "\r\n--------------------------\r\nSent wirelessly by AvocadoIT from my BlackBerry Handheld",
    '\r\nThank you\r\nBrian\r\nBrian Robinson\r\nTechnical Support Engineer\r\n408 562 7952 (W)\r\n408 857 8740 (C)\r\n2211 N. 1st. Suite 200,\r\nSan Jose, Ca 95131\r\n"Sent wirelessly from AvocadoIT - the leading enterprise mobility company"',
    "\r\nJS\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!",
    "--------------------------\r\nSent wirelessly from AvocadoIT, the leader in mobile wireless platforms for the Corporate market\r\n-----------------------------",
    "\r\nSent wirelessly from AvocadoIT, Inc.",
    '\r\nBrian Robinson\r\nTechnical Support Engineer\r\n408 562 7952 (W)\r\n408 857 8740 (C)\r\n2211 N. 1st. Suite 200,\r\nSan Jose, Ca 95131\r\n"Sent wirelessly from AvocadoIT - the leading enterprise mobility company"',
    "\r\nJanie\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!",
    "Sent wirelessly from AvocadoIT, Inc. \r\nPrakash Iyer\r\nFounder, CTO & Sr. Vice President, Engineering\r\nDirect: 408.562.8030 \r\npiyer@avocadoit.com ",
    "\r\nPrakash\r\nPrakash Iyer\r\nFounder, CTO & Sr. Vice President, Engineering\r\nDirect: 408.562.8030\r\npiyer@avocadoit.com\r\n",
    "Sent wirelessly from AvocadoIT, Inc.",
    "\r\n--------------------------\r\nSent wirelessly from AvocadoIT",
    "--------------------------\r\nSent wirelessly from AvocadoIT - the leading enterprise mobility company",
    "\r\n------------------------------------------------------\r\nSent wirelessly from AvocadoIT!",
    "Sent wirelessly from AvocadoIT - The leading Enterprise Mobility Company\r\nDavid C. Meltzer, Esq.  Director of Sales AvocadoIT",
    "\r\n---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\nThanx,\r\nDan.\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com\r\n",
    "\r\nThanx,\r\nDan.\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.562.8041, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com",
    "\r\n--- \r\nDan Baca                                                dbaca@avocadoit.com \r\nDirector of Sales Engineering                   408.394.6114 m \r\nwww.avocadoit.com                                       408.562.8041 w",
    "\r\n---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    "\r\n--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---\r\nGet mobilized with AvocadoIT  (www.avocadoit.com)",
    "\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.562.8041, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com",
    "\r\n---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057, c 408.394.6114\r\nAvocadoIT                                 www.avocadoit.com",
    "---\r\nDan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering\t\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "--- \r\nDan Baca                                                dbaca@avocadoit.com \r\nDirector of Sales Engineering                   408.394.6114 m \r\nwww.avocadoit.com                                       408.562.8041 w",
    "\r\nDan Baca \t\t\tdbaca@avocadoit.com\t\t408.562.8041\r\nDirector of Engineering\t\twww.avocadoit.com\t\t408.394.6114 m",
    "Dan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Sales Engineering \t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\n--- \r\nDan Baca                                              dbaca@avocadoit.com \r\nDirector of Technical Marketing               408.394.6114 m \r\nwww.avocadoit.com                               408.562.8041 w \r\n",
    "\r\n--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---\r\nSent from my AvocadoIT mobile device (www.avocadoit.com)",
    "\r\n---\r\nRicardo Garcia                   \t\t\tricardo.garcia@avocadoit.com\r\nTechnical Marketing Engineer            \t408.829.4827 mobile\r\nwww.avocadoit.com                           \t408.562.7984 work",
    "\r\nCraig Stewart\r\nVice President\r\nGlobal Telecom Business Unit\r\n________________________________\r\nAvocadoIT\r\n3231 Scott Boulevard\r\nSanta Clara, California 95054\r\nOffice:    408-562-8060\r\nFax:       408-562-8100\r\nMobile:   510-409-7173\r\nWeb:      www.avocadoit.com",
    "---\r\nDan Baca                                  dbaca@avocadoit.com\r\nDirector of Engineering               408.588.4057\r\nAvocadoIT                                 www.avocadoit.com",
    "--------------------------\r\nDan Baca                                 dbaca@avocadoit.com\r\nDirector of Sales Engr              408-394-6114 m\r\nAvocadoIT Inc.                         408-562-8041 w\r\n---  www.avocadoit.com",
    "Dan Baca\t\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Product Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t\t408.562.8041 w",
    "\r\n--- \r\nDan Baca                                              dbaca@avocadoit.com \r\nDirector of Technical Marketing               408.394.6114 m \r\nwww.avocadoit.com                               408.562.8041 w",
    "Dan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    "---\r\nDan Baca\t\t\t\t\tdbaca@avocadoit.com\r\nDirector of Technical Marketing\t\t\t408.394.6114 m\r\nwww.avocadoit.com\t\t\t\t408.562.8041 w",
    "\r\n_______________________________\r\nReporting Division\r\nCustomer Service Dept.\r\nWebEx Communications, Inc.\r\nReports@webex.com\r\n1.408.435.7088 option #5 \r\nwww.webex.com\r\nBringing the meeting to you.\r\n",
    "\r\nEric Shurtleff\r\nCorporate Account Manager\r\nWebEx Communications, Inc.\r\n* 408.435.7253\r\nTech Support:          support@webex.com;   408-435-7088\r\nCustomer Service:    customer_services@webex.com;    408-435-7088\r\n",
    
    "\r\nJennifer Kouragian\r\nAdministrative Support\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8012\r\nMobile: 408.420-4046\r\nFax: 408.562.8100 \r\njennifer.kouragian@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    '\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile',
    "r\nAmy Moore\r\nExecutive Assistant\r\nMark Tapling, President & CEO\r\nMatt DiMaria, Sr. VP Marketing & Business Development\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8106  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO &\r\nExecutive Assistant to Matt DiMaria, Sr. VP of Business Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",    
    "\r\nThank you!\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks!\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO \r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nA \r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131 \r\ntel: (408) 562-8032\r\nfax: (408) 562-8101\r\nmobile: (408) 857-8520",
    "\r\nThanks,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO \r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street\r\nSuite 200\r\nSan Jose, CA  95131",
    "\r\nThank you,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131\r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile\r\n",
    "\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla, CEO \r\nAvocadoIT, Inc. \r\n2211 North First Street, Ste 200 \r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel \r\n(408) 562-8108  fax \r\n(408) 857-8520  mobile\r\n",
    "\r\nThanks all,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO &\r\nExecutive Assistant to Matt DiMaria, Sr. VP of Business Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla, CEO \r\nAvocadoIT, Inc. \r\n2211 North First Street, Ste 200 \r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel \r\n(408) 562-8108  fax \r\n(408) 857-8520  mobile \r\n",
    "\r\nThanks,\r\nAmy\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla, CEO \r\nAvocadoIT, Inc. \r\n2211 North First Street, Ste 200 \r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel \r\n(408) 562-8108  fax \r\n(408) 857-8520  mobile",
    "\r\nThanks,\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks so much,\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks again,\r\nAmy \r\nExecutive Assistant to Venk Shukla\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd\r\nSanta Clara, CA  95054\r\ntel: (408) 562-8032\r\nfax: (408) 562-8100",
    "\r\nThanks,\r\nNoemi\r\nExecutive Assistant to Venk Shukla",
    "\r\nThanks,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO\r\nAvocadoIT, Inc.\r\n2211 North First Street, Ste 200\r\nSan Jose, CA  95131\r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile\r\n",
    "\r\nThank you,\r\nAmy\r\nAmy Moore\r\nExecutive Assistant to Venk Shukla, CEO \r\nand Matt DiMaria, Sr. VP Business Operations\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\n(408) 562-8032  tel\r\n(408) 562-8108  fax\r\n(408) 857-8520  mobile",
    "\r\nAmy Moore \r\nExecutive Assistant to Venk Shukla \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA  95131  \r\ntel: (408) 562-8032 \r\nfax: (408) 562-8100",
    "\r\nGermana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nGermana Martinez\r\nHuman Resources\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez\r\nHuman Resources Coordinator and\r\nSales Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nGermana Martinez\r\nHuman Resources Coordinator,\r\nSales Executive Admin and\r\nAssistant to the CFO\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez\r\nRecruiting Coordinator and\r\nExecutive Admin for Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nGermana Martinez \r\nHuman Resources Coordinator, \r\nand Executive Admin \r\nAvocadoIT, Inc. \r\n2211 North First St. Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-8135 Phone \r\n408-562-8107 Fax",
    "\r\nGermana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408-562-8135 PH\r\n408-562-8100 FX",
    "Germana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "Germana Martinez\r\nHuman Resources\r\nAdministrative Services Coordinator,\r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez \r\nHuman Resources \r\nAdministrative Services Coordinator, \r\nAvocadoIT, Inc. \r\n2211 North First St. Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-8135 Phone \r\n408-562-8107 Fax ",
    "\r\nGermana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8100 FX",
    "Germana Martinez\r\nRecruiting Coordinator\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408-562-8135 PH\r\n408-562-8100 FX",
    "Germana Martinez\r\nHuman Resources Coordinator and\r\nSales Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "Germana Martinez\r\nHuman Resources Coordinator,\r\nand Executive Admin \r\nAvocadoIT, Inc.\r\n2211 North First St. Suite 200\r\nSan Jose, CA 95131\r\n408-562-8135 Phone\r\n408-562-8107 Fax",
    "\r\nGermana Martinez \r\nAdministrative Services Coordinator, \r\nAvocadoIT, Inc. \r\n2211 North First St. Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-8135 Phone \r\n408-562-8107 Fax ",
    "\r\nThanks,\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSteven Hirata\r\nCEO\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect - 408-562-8065\r\nshirata@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nRegards,\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n(408)562-8065",
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644",
    "Mike Richman\r\nSenior Account Manager\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\noffice (forwards to cell phone): 415-460-0644\r\nMike.Richman@AvocadoIT.com",
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North first Street, Suite 200\r\nSan Jose, CA  95131\r\n415-460-0644 office\r\n408-205-9961 cell\r\n415-460-0722 fax",
    "\r\nMike Richman\r\nSenior Account Executive\r\n2211 North first Street, Suite 200\r\nSan Jose, CA  95131\r\n415-460-0644 office\r\n408-205-6691 cell\r\n415-460-0722 fax",
    "\r\nThanks\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT\r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT\r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT\r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nThanks.\r\nStuart\r\n-------------------------------------------\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT, Inc.\r\nstuart.finn@avocadoit.com\r\n408-562-7950\r\n--------------------------------------",
    "\r\nThanks.\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales & Services\r\nAVOCADOIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nStuart.Finn@AvocadoIT.com\r\n408-562-7950\r\nwww.AvocadoIT.com",
    "\r\nThanks.\r\nStuart\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nThanks.\r\nStuart Finn\r\nSenior Vice President, Global Sales and Service\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\nstuart.finn@avocadoit.com\r\n408-562-7950",
    "\r\nThanks.\r\nStuart Finn\r\nSenior Vice President,\r\nSales and Service\r\nAVOCADOIT, INC.\r\n2211 N. First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n408-562-7950 (Office) \r\nstuart.finn@avocadoit.com \r\nwww.avocadoit.com",
    '\r\n--------------------------\r\nrm -Rf //taliban/bin/laden',
    '--------------------------\r\nrm -Rf //taliban/bin/laden',
    '\r\n"rm -Rf /bin/laden"\r\nCraig',
    '\r\nCraig',
    '\r\nDoug Field',
    '\r\nDouga....',
    '\r\ndouga...',
    '\r\nJohn',
    '\r\nSrik',
    '\r\nThanks\r\nPrakash',
    '\r\nMahesh',
    '\r\nOm',
    '\r\n-Himmat',
    '\r\nShailesh',
    '\r\nRajeev',
    '\r\nRavi',
    '\r\nSrikanth',
    '\r\nThanks,\r\nSriram',
    '\r\n--Alex',
    '\r\n-Tony',
    '\r\nRayhan\r\n--------------------------\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com',
    '\r\n--------------------------\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com',
    "\r\n--------------------------\r\nRayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.",
    "\r\nDion \r\n----------\r\nDion Bozec, B. Eng\r\nAerospace Engineer\r\nMxi Technologies\r\ndbozec@mxi.com\r\n(613) 747-4698 x266\r\n",
    "\r\nRegards,\r\nTony\r\nAnthony N. Tarsia\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n4094 Majestic Lane, PMB 127\r\nFairfax, VA 22033\r\nOffice - 703-273-6313      Cell - 703-861-5292\r\nFax - 703-995-4722          E-mail - anthony.tarsia@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nRayhan",
    "\r\nRicardo",
    "\r\nRegards, \r\nVinay Settepalli \r\nSr.Software Engineer, \r\nFrance Telecom R&D,LLC, \r\n1000 Marina Blvd,Suite #300, \r\nBrisbane,CA-94005. \r\n",
    "Rayhan bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com",
    "\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nBrought to you wirelessly by AvocadoIT Inc.",
    "\r\nJon LaPorte\r\nNW Sales Director\r\nAvocadoIT, Inc.\r\n7041 23rd Avenue NE\r\nSeattle, WA 98115\r\nDirect: 206.985.2040\r\nFax: 309.216.9163\r\njon.laporte@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin\r\n",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/> \r\nWhere insiteful promotions begin",
    'Marcia Kadanoff\r\nChief Marketing Officer\r\n408.861.9242 direct\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin\r\n',
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin\r\n\xa0\r\n",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk \r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\n--\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n650.270.4309 cell/messages\r\n<<Sent Wirelessly using OmniSky>>\r\n",
    "Marcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin\r\nIn an emergency - page me at mkadanoff.pager@iq.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\n408.861.9242 direct\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\n408.777.4040 fax\r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk                408.777.4040 fax\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "\r\nMK\r\n\xa0\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 408.777.4040 fax\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk \r\nmkadanoff@iq.com <mailto:mkadanoff@iq.com> \r\nwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin\r\n",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/>\r\nWhere insiteful promotions begin\r\n",
    "\r\n\xa0\r\nMK\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/>\r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n408.562.8050 direct\r\n650.270.4309 mobile",
    "\r\n\xa0\r\nBest regards,\r\n\xa0\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\n--\r\nMarcia Kadanoff\r\n<<Sent Wirelessly using OmniSky>>\r\n",
    "iQ.COM\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\niQ.COM\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "Marcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin",
    "Marcia Kadanoff\r\nChief Marketing Officer\r\n408.861.9242 direct\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\nAvocadoIT \r\n408.562.8050 - new direct dial #! \r\nwww.avocadoit.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT\r\n408.562.8050 - new direct dial #!\r\nwww.avocadoit.com",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMK\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\n650.343.4309 talk\r\n650.343.2042 fax\r\nmarciak@msdirect.com\r\nwww.msdirect.com\r\n",
    "\r\n\xa0\r\nmk\r\nMarcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n2211 North First Street, \r\nSuite 200, San Jose,\r\nCA  95131\r\nDirect: 408.562.8035 \r\nFax: 408.562.8103\r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRitesh Adval\r\nSoftware Engineer\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nThanks\r\nRitesh\r\nRitesh Adval\r\nAvocadoIT, Inc.\r\n2211 North First Street,\r\nSuite 200, San Jose,\r\nCA  95131\r\nDirect: 408.562.8035\r\nFax: 408.562.8103\r\nradval@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\nLisa Zissen  \r\nExecutive Assistant to Prakash Iyer\r\nFounder, CTO and Sr. VP of Engineering Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen\r\nExec. Asst. to Prakash Iyer, CTO & Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nThanks!\r\nLisa\r\nLisa Zissen\r\nExecutive Assistant to Prakash Iyer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite #200\r\nSan Jose, California  95131\r\n408.562.8012 - direct\r\n408.205.0665 - Cell\r\nlisa.zissen@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nLisa Zissen \r\nExecutive Assistant to Prakash Iyer \r\nFounder, CTO and Sr. VP of Engineering Operations \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\n2nd Floor \r\nSan Jose, California  95131 \r\n(408) 562-8012 - direct \r\n(408) 205-0665 - Mobile \r\n(408) 562-8100 - Fax \r\nlisa.zissen@avocadoit.com \r\nwww.avocadoit.com \r\nAvocadoIT, your business everywhere! \r\n_____  \r\nUpgrade Your Email - Click here!",
    "\r\nRay Rahamin\r\nSent Wirelessly form AvocadoIT -- The Leading Enterprise Mobility Company",
    "\r\nRay Rahamin\r\nGeneral Manager\r\nFinancial Services Industry\r\nSent Wirelessly from AvocadoIT -- The Leading Enterprise Mobility Company",
    "\r\nRay Rahamin\r\nGeneral Manager\r\nFinancial Services Industry\r\nAvocadoIT\r\nOffice: 914-773-0593\r\nCell   : 914-645-6617\r\nFax  : 914-773-0595\r\nRay.Rahamin@avocadoit.com",
    "\r\nRay Rahamin\r\nGeneral Manager\r\nFinancial Services Industry!",
    "\r\nThanks, \r\nMehrak Hamzeh\r\nMobility Consultant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100 \r\nhttp://www.avocadoit.com",
    '\r\nMehrak Hamzeh\r\nMobility Consultant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100\r\nAre you ready for the wireless web? \r\nhttp://www.avocadoit.com',
    "\r\nMehrak Hamzeh\r\nMobility Consultant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100 \r\nhttp://www.avocadoit.com",
    "\r\nMehrak Hamzeh\r\nMobility Consultant\r\nwww.avocadoit.com\r\n3231 Scott Blvd. Santa Clara, CA 95054\r\nPhone: 408.562.7989\r\nFax:408.562.8100",
    "\r\nRay Rahamin\r\nGeneral Manager of Financial Services Industry \r\nOffice:  914.773.0593\r\nCell:      914.645.6617\r\nfax:  914.773.0595",
    "\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 office\r\n408.562.8100 fax\r\n408.420.3841 mobile\r\nryan.glover@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nAjay Karwal\r\nakarwal@avocadoit.com\r\nwww.avocadoit.com\r\n408-562-8111",
    "\r\nThanks,\r\najay\r\nAjay Karwal\r\nApplication Architect\r\nakarwal@avocadoit.com\r\nwww.avocadoit.com\r\n408-562-8111(Direct)\r\n408-420-3854(Cell)",
    "\r\nAjay Karwal\r\nManager, Application Engineering\r\nakarwal@avocadoit.com\r\nwww.avocadoit.com\r\n408-562-8111(Direct)",
    "McCann\r\nAvocadoIT, Inc\r\n8745 W. Higgins, Suite 360\r\nChicago, IL 60631\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225.McCannl@avocadoit.com\r\nwww.avocadoit.com\r\n--------------------------\r\nSent from my Wireless Handheld",
    "McCann\r\nAvocadoIT, Inc\r\n408-562-7951\r\n--------------------------\r\nSent from my Wireless Handheld",
    "\r\nThanks,.McCann@avocadoit.com\r\nAvocadoIT, Inc.\r\n408-562-7951",
    "McCann, Central Region Director\r\nAvocadoIT, Inc\r\nOffice:  773.243-0324\r\nMobile: 847-826-8064\r\nFax: 773-243-0225.McCann@avocadoit.com\r\nwww.avocadoit.com\r\n---------------------",
    'McCann, District Manager\r\nAvocadoIT, Inc.\r\n8735 W. Higgins, Suite 235\r\nChicago, IL 60631\r\nOffice 773-243-0324\r\nFax  773-243-0225\r\nwww.AvocadoIT.com\r\n"AvocadoIT, Your Business Everywhere!"',
    "\r\nNihar Mehta\r\nAvocadoIT Inc.\r\n408-562-8019",
    "\r\n-------------------------------\r\nSent Wirelessly from AvocadoIT - the Leading Enterprise Mobile Company",
    "\r\n________________________\r\nChris Longstaffe\r\nDirector of Training\r\nAvocadoIT, Inc\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8132\r\nMobile: 408.205.7573\r\nFax: 408.562.8014\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nChris\r\n--------------------------\r\nChris Longstaffe\r\n(408) 205 7573\r\nSent using my AvocadoIT wireless device\r\nAvocadoIT ... Your Business, Everywhere!",
    "\r\nThanks\r\nChris\r\n________________________\r\nChris Longstaffe\r\nDirector of Training, AvocadoIT\r\nTel: (408) 562 8132\r\nCell: (925) 640 3166",
    "\r\nthanks\r\nChris\r\n_______________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc\r\n2211 North First St, Suite 200\r\nSan Jose, CA 95131\r\nDirect: (408) 562 8132\r\nMobile: (408) 205 7573\r\nFax: (408) 562 8104\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks\r\nChris\r\n________________________\r\nChris Longstaffe\r\nDirector of Training, AvocadoIT\r\nTel: (408) 562 8132\r\nCell: (408) 205 7573",
    "\r\nChris\r\n________________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nOffice: (408) 562 8132\r\nMobile: (408) 205 7573",
    "\r\nThanks\r\nChris\r\n________________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nOffice: (408) 562 8132\r\nMobile: (408) 205 7573",
    "\r\nChris\r\n________________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nOffice: (408) 562 8132\r\nMobile: (408) 205 7573",
    "\r\nThanks\r\nChris\r\n_______________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc\r\n2211 North First St, Suite 200\r\nSan Jose, CA 95131\r\nDirect: (408) 562 8132\r\nMobile: (408) 205 7573\r\nFax: (408) 562 8104\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n_______________________\r\nChris Longstaffe\r\nSenior Director, Education Services\r\nAvocadoIT, Inc\r\n2211 North First St, Suite 200\r\nSan Jose, CA 95131\r\nDirect: (408) 562 8132\r\nMobile: (408) 205 7573\r\nFax: (408) 562 8104\r\nchris.longstaffe@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n--------------------------\r\nChris Longstaffe\r\n(408) 205 7573\r\nSent using my AvocadoIT wireless device\r\nAvocadoIT ... Your Business, Everywhere!",
    "\r\nChris\r\n________________________\r\nChris Longstaffe\r\nMobile: (925) 640 3166\r\nchris@integratis.com",
    "Thanks.\r\nChris Longstaffe\r\n(408)205 7573",
    "Scott\r\n--------------------------\r\nSent Wirelessly from AvocadoIT - the leading enterprise mobility company\r\nScott Weller\r\n206-323-5105 (o)\r\n206-604-6105 (m)",
    "\r\nScott Weller\r\nSenior Director - Telecom\r\nAvocadoIT, Inc\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\n206-323-4314 (f)\r\nwww.avocadoit.com",
    "\r\nScott\r\nScott Weller\r\nDirector, Business Development - Telecom\r\nAvocadoIT, Inc\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\n206-323-4314 (f)\r\nwww.avocadoit.com",
    "\r\nScott Weller\r\nDirector, Business Development - Telecom\r\nAvocadoIT, Inc\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\n206-323-4314 (f)\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nScott\r\nScott Weller\r\nSr. Director - Telecom\r\nAvocadoIT, Inc\r\n(o)  206.323.5105\r\n(m)  206.604.6105\r\n(f)  206.323.4314",
    "\r\nRegards,\r\nScott\r\nScott Weller\r\nSr. Director - Telecom\r\nAvocadoIT, Inc\r\n(o)  206.323.5105\r\n(m) 206.604.6105\r\n(f)   206.323.4314",
    "\r\nScott\r\nScott Weller\r\nSr. Director - Telecom\r\nAvocadoIT, Inc\r\n(o)  206.323.5105\r\n(m)  206.604.6105\r\n(f)  206.323.4314",
    "\r\nKhou Fang\r\nTechnical Recruiting Manager\r\nDirect: 408.562.8096\r\nMobile: 408.829-4796\r\nFax: 408.562.8107",
    "\r\nScott Weller\r\nDirector - Telecom Business Development\r\nAvocadoIT\r\n206-323-5105 (o)\r\n206-604-6105 (m)\r\nhttp://www.avocadoit.com",
    '\r\n__________________________________________________\r\nDo You Yahoo!?\r\nYahoo! Photos - 35mm Quality Prints, Now Get 15 Free!\r\nhttp://photos.yahoo.com/\r\n',
    '\r\nRegards\r\n________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com\r\n',
    '\r\n______________________________________________________\r\nGet Your Private, Free Email at http://www.hotmail.com\r\n',
    '\r\n_________________________________________________________________\r\nGet your FREE download of MSN Explorer at http://explorer.msn.com\r\n',
    '\r\n_________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com.\r\nShare information about yourself, create your own public profile at \r\nhttp://profiles.msn.com.\r\n',
    '\r\n*********************************\r\nR. Scott Isgitt\r\nFormer Director of Sales\r\nAvocadoIT Inc.\r\n512-331-5459 - Home\r\n512-636-1275 - Mobile\r\nscottisgitt@hotmail.com\r\n*********************************',
    '\r\n(408)297-7836\r\n(408)996-8425\r\n(408)255-2726\r\n(408)297-4544\r\n(408)271-0826\r\n(650)917-1424\r\nMichelle\r\n________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com\r\n',
    '\r\nBLJ\r\n________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com\r\n',
    '\r\nLuong\r\n________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com\r\n',
    '\r\n________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com\r\n',
    ' N Mahabhashyam\r\n________________________________________________________________________\r\nGet Your Private, Free E-mail from MSN Hotmail at http://www.hotmail.com\r\n',
    'AvocadoIT, Your Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.562-8100 \r\nwww.avocadoit.com \r\n*************************************************'
    'AvocadoIT, Your Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************',
    "AvocadoIT, Your Business  Everywhere\r\n***********************************************\r\nRon Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc\r\n2211 N First Street, Suite 200\r\nSan Jose CA, 95131\r\nw 408.562-8039\r\nm 415.845-6789\r\nf 408.562-8100\r\nwww.avocadoit.com\r\n*************************************************",
    "\r\nVenktesh Shukla\r\nChief Executive Officer\r\nAvocadoIT, Inc.\r\n2211 No. First Street, Ste. 200\r\nSan Jose, CA  95131\r\nDirect:   (408) 562-8010\r\nFax:      (408) 562-8108\r\nwww.avocadoit.com",
    "\r\n--\r\nAvocadoIT Europe GmbH\r\nHamborner Strasse 53\r\n40472 Duesseldorf, Germany\r\nphone\t+49 (0) 211.94098-187\r\nfax\t+49 (0) 211.94098-200\r\ncellular\t+49 (0) 171.3845255\r\nmailwww.avocadoit.com",
    "Stewart\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nMobile:   408-891-8139\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    craig.stewart@avocadoit.com\r\nWeb:      www.avocadoit.com",
    "\r\nGiselle Bisson\r\nDirector of Corporate Communications\r\navocadoit\r\n415-608-7406 cell\r\n408-588-4057 work\r\nhttp://www.avocadoit.com'",
    "Steven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "Stewart\r\nVP - Global Telecom Business Unit\r\nAvocadoIT\r\n3231 Scott Boulevard\r\nSanta Clara CA 95054\r\n408-562-8060 office\r\n408-562-8100 fax\r\n510-409-7173 mobile\r\nwww.avocadoit.com\r\n",
    "Grant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n678.522.8238  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\nthanks,\r\nPaiman Komeilizadeh\r\nAccounting Manager\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain:  408.562.8000\r\nDirect: 408.562.8053\r\nFax:     408.562.8100\r\npkomeili@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nGiselle Bisson\r\nDirector of Corporate Communications\r\navocadoit\r\n415-608-7406 cell\r\n408-588-4057 work\r\nhttp://www.avocadoit.com",
    "\r\n--------------------------\r\nSent from my RIM pager - via AvocadoIT (www.avocadoit.com)",
    "Iyer\r\nFounder & CTO\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8030 \r\nFax: 408.562.8100 \r\npiyer@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nThanks\r\nPhilip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\nDavid\r\nDavid Marcus\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRegards,\r\nRay\r\n------------------------------------------------------------------------------------------------------\r\nRay Canuel\t\t\trcanuel@avocadoit.com\t\t\tdbaca@avocadoit.com\r\nSales Engineer\t\t\t703.281.4223  W\r\nwww.avocadoit.com                                703.728.2007  M\r\nSent from my AvocadoIT Wireless Handheld",
    "\r\nHimmat Vansdadia / CCIE # 4864\r\nSr. Network Engineer\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8045 \r\nMobile: 408-590-8883\r\nFax: 408.562.8100 \r\nhvansdadia@avocadoit.com \r\nwww.avocadoit.com",
    "\t\t\tricardo.garcia@avocadoit.com\r\nSales Engineer\t\t\t408.829.4827 m\r\nwww.avocadoit.com\t\t408.562.7984 w",
    "\r\nL\r\nLaura C. Henderson\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\nemail: lhenderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nhttp://www.avocadoit.com",
    "\r\nThanks,\r\nBarry\r\nBarry Phillips\r\nDirector, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-8016 (direct)\r\n(408) 562-8100 (fax)\r\nwww.avocadoit.com",
    "Stewart\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, California 95054\r\nMobile:   408-891-8139\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    cstewart@avocadoit.com\r\nWeb:      www.avocadoit.com",
    "\r\n---  Garcia \r\nSales Engineer \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\n(408) 829-4827 cell \r\n(408) 562-7984 office \r\n(408) 562-8100 fax \r\nricardo.garcia@avocadoit.com \r\nwww.avocadoit.com",
    "\r\n*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*\r\nYUMIKO KAWAKAMI\r\nyumiko.kawakami@avocadoit.com\r\nAvocadoIT Japan K.K.\r\nKamiyacho MT Bldg. 14th Floor 4-3-20 Toranomon,\r\nMinato-ku, Tokyo 105-0001 JAPAN\r\nTEL : 3-5404-3430(direct)/FAX : 3-5404-3401\r\nwww.avocadoit.com/jp\r\n*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*",
    "\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice President - Business Development\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "\r\nKumi Thiruchelvam\r\nAvocadoIT Europe\r\n100 Pall Mall\r\nSt James'\r\nLondon\r\nSW1Y 5HP\r\nTel. +44 20 7664 8610\r\nFax. +44 20 7664 8611\r\nMobile +44 7802 150136\r\nkumi@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\njk\r\n--------------------------------------\r\nJimmy Kim\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\nOffice: 312-409-4919\r\nFax: 810.279.4299\r\njimmy.kim@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nSusan Simmons\r\nDirector of Finance & Administration and Corporate Controller\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8022\r\nMobile: 408.393.3167\r\nFax: 408.562.8100\r\nsusan.simmons@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRay\r\nRay Chernaskey\r\nDistrict Sales Manager - East\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA  95131 \r\nray.chernaskey@avocadoit.com\r\nOffice: 610-367-7130 \r\nFax: 610-367-7230 \r\nwww.avocadoit.com",
    "\r\nThanks!\r\nLaura\r\nLaura C. Henderson\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\nemail: lhenderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nhttp://www.avocadoit.com",
    "\r\nThanks!\r\n-Lisa\r\nLisa Crowder\r\nExec. Asst. to Prakash Iyer, CTO & Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.crowder@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nJoe Hong\r\nAvocadoIT Canada Corp. \r\nhttp://www.avocadoit.com\r\nEmail: joe.hong@avocadoit.com\r\nPhone: 416-643-4864\r\nMobile: 416-346-2729 \r\n_________________________________________________________________\r\nClick to add my contact info to your organizer: \r\nhttp://my.infotriever.com/jhong",
    "\r\nThanks,\r\nDoug Adkins\r\nDirector, Technical Support\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, Ca. 95131\r\nOffice:(408) 562-7985\r\nMobile: (408) 829-5118\r\nFax: (408) 562-8100\r\nEmail: doug.adkins@avocadoit.com\r\n<< OLE Object: Picture (Metafile) >> \r\nwww.avocadoit.com\r\n",
    "\r\nThanks,\r\nBithi\r\nBithi Agrawal\r\nProject Manager\r\nAvocadoIT, Inc.\r\n2211 N. First Street, Suite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8071\r\nFax: 408.562.8100\r\nbagrawal@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nNobuyuki Takaki\r\nDirector of Business Development\r\nMail: nobuyuki.takaki@avocadoit.com\r\nDI: 81-3-5404-3418\r\nHP: www.avocadoit.com",
    "\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "Stewart\r\nVice President\r\nGlobal Telecom Business \r\n____________________________\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, California 95054\r\nOffice:    408-562-8060  \r\nFax:       408-562-8100\r\nEmail:    cstewart@avocadoit.com\r\nWeb:      www.avocadoit.com",
    "\r\nThanks,\r\nNilesh\r\n___________________________________________\r\nNilesh Bodade\r\nApplications Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8126\r\nFax: 408.562.8100\r\nnbodade@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nthanks\r\nPaiman Komeilizadeh\r\nAccounting Manager\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain:  408.562.8000\r\nDirect: 408.562.8053\r\nFax:     408.562.8100\r\npkomeilizadeh@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nBrandi Bernazzani\r\nAvocadoIT\r\nAccount Development Representative\r\n(800) 355-1103 ext. 8188\r\nbrandib@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nLisa Crowder\r\nAssistant to the CTO and VP of Engineering\r\nAvocadoIT, Inc.\t\t\t\t\t\r\n2211 N. First Street, Suite #200\r\nSan Jose, California  95131\r\nPhone - (408) 562-8012\r\nFax - (408) 562-8103\r\nwww.avocadoit.com",
    "Nobuyuki Takaki\r\nDirector of Business Development\r\nMail: nobuyuki.takaki@avocadoit.com\r\nDI: 81-3-5404-3418\r\nHP: www.avocadoit.com",
    "Doug Field\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: \r\nFax: 408.562.8100 \r\ndouglas.field@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nBarry\r\nBarry Phillips\r\nDirector of Solutions Marketing\r\nTravel and Transportation \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-8016 (direct)\r\n(408) 562-8100 (fax)\r\nwww.avocadoit.com",
    "Steven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nChris\r\nChristopher Quinter\r\nAccount Manager\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Office\r\n(215) 778-3235  Mobile\r\nMobilizing solutions to reduce costs, increase revenue & improve productivity www.avocadoit.com",
    "\r\nKelsey\r\nPlease note -- NEW ADDRESS BELOW!!!\r\nKelsey Kerr\r\nAccount Manager\r\nAvocadoIT\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408.562.8036 (office)\r\n408.394.1054 (cell)\r\nkkerr@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nKlee Larsen\r\nklarsen@avocadoit.com\r\nSr. Technical Writer\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000\r\nDirect: 408.562.8062\r\nFax: 408.562.8100\r\nwww.avocadoit.com",
    "\r\nPhilip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\n---\r\nGeoff Vona\r\nDirector R&D\r\nAvocadoIT Canada\r\n49 Ontario Street\r\nBox 24, 6th Floor\r\nToronto ON M5A 2V1\r\nMain: 416.366.6425\r\nFax: 416.643.4832\r\ngeoff.vona@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRegards,\r\nPhilip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 212-666-1194\r\nhttp://www.avocadoit.com",
    "\r\nLaura C. Henderson\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\nemail: lhenderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nhttp://www.avocadoit.com",
    "\r\nAmit Vyas\r\nNetworks and Operations Support Engineer\r\nAvocadoIT, Inc.\r\nDirect: 408.562.8114 \r\nFax: 408.562.8100 \r\navyas@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nAll the best , Stewart \r\nVice President \r\nGlobal Telecom Business \r\n____________________________ \r\nAvocadoIT, Inc. \r\n3231 Scott Boulevard \r\nSanta Clara, California 95054 \r\nMobile:   510-409-7173 \r\nOffice:    408-562-8060  \r\nFax:       408-562-8100 \r\nEmail:    cstewart@avocadoit.com \r\nWeb:      www.avocadoit.com",
    "\r\nRichard Rapaport\r\n415.387.4751  \r\nRJRap@aol.com\r\nThank you,\r\nRyan\r\n____________________________\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 office\r\n408.562.8100 fax\r\n408.420.3841 mobile\r\nryan.glover@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n*New Address*\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nSrilatha Seethamraju,\r\nApplications Engineer,\r\nAvocadoIT Inc.,\r\n2211 N.First Street,\r\nSan Jose, CA  95131.\r\n408.562.8073 - Direct Dial\r\n408.562.8000 - Corporate\r\n408.562.8104 - Fax\r\nsrilatha.seethamraju@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThank you!\r\n-Lisa\r\nLisa Crowder\r\nExecutive Assistant to Prakash Iyer, C.T.O. and Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, CA  95131\r\n(408) 562-8012 - Direct\r\n(408) 562-8103 - Fax\r\nlisa.crowder@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nThanks,\r\ndave\r\n--\r\nDave Truman  (416.366.6425 x732)            dave.truman@avocadoit.com\r\nTechnical Publications                4162002292@txt.bellmobility.ca\r\nAvocadoIT Canada        http://epiwww.avocadoit.com/mobility/rd/pubs\r\n_____________________________________________________________________\r\n/** brevis esse laboro, obscuris fio.  \r\n(I try to be brief, but only become more obscure.)      <Homer> */",
    "\r\nThanks\r\nStephen T. Ng\r\nSr. Technical Support Engineer\r\n2211 N. First Street, Suite# 200\r\nSan Jose, Ca 95131\r\nOffice - 408-562-7951\r\nCell    - 408-857-8739\r\nEmail - stephen.ng@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nJoe Hong\r\nAvocadoIT Canada Corp.\r\nhttp://www.avocadoit.com\r\nEmail: joe.hong@avocadoit.com \r\nPhone: 416-643-4864\r\nMobile: 416-346-2729\r\n_________________________________________________________________\r\nClick to add my contact info to your organizer: \r\nhttp://my.infotriever.com/jhong\r\n",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nBest regards,\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 office\r\n408.562.8100 fax\r\n408.420.3841 mobile\r\nryan.glover@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nThanks, \r\nDavid Marcus\r\nDirector\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nJoe Hong\r\nAvocadoIT Canada Corp.\r\nhttp://www.avocadoit.com\r\nEmail: joe.hong@avocadoit.com\r\nPhone: 416-643-4864\r\nMobile: 416-346-2729\r\n_________________________________________________________________\r\nClick to add my contact info to your organizer:\r\nhttp://my.infotriever.com/jhong\r\n",
    "Stewart \r\nVice President \r\nGlobal Telecom Business \r\n____________________________ \r\nAvocadoIT, Inc. \r\n3231 Scott Boulevard \r\nSanta Clara, California 95054 \r\nMobile:   510-409-7173 \r\nOffice:    408-562-8060  \r\nFax:       408-562-8100 \r\nEmail:    cstewart@avocadoit.com \r\nWeb:      www.avocadoit.com",
    "\r\nDavid Chan\r\nVP Business Development\r\nAvocadoIT\r\n408-588-4057 (Office)\r\n415-819-5490\r\nwww.avocadoit.com\r\nMobilizing your web content for wireless phones, PDAs, pagers and regular voice phones",
    "\r\nBrett \r\nBrett W. Welborn \r\nAccount Development \r\nAvocadoIT, Inc. \r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n(800) 355-1103 ext. 8186 \r\n(408) 562-8186 (direct)",
    "\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n*New Address* \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nScott Bermon\r\nTeam Lead, Alerts Platform\r\nAvocadoIT Canada\r\n49 Ontario Street\r\nBox 24, 6th Floor\r\nToronto ON M5A 2V1\r\nMain: 416.366.6425 x328\r\nMobile: 416.986.4685\r\nEmail: scott.bermon@avocadoit.com\r\nWeb: www.avocadoit.com",
    "Stewart\r\nVice President\r\nGlobal Telecom Business Unit\r\n408-562-8060 office\r\n408-562-8100 fax\r\nwww.avocadoit.com\r\n",
    "\r\nDave\r\nDavid C. Meltzer\r\nSales Director - Southwest\r\nAvocadoIT, Inc.\r\nPhone - 858-547-3567\r\nCell     - 858-774-0033\r\nemail - david.meltzer@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nDonivan Fox\r\nManager, Applications Engineering\r\nAvocadoIT, Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nMain: 408.562.8000 \r\nDirect: 408.562.8001\r\nMobile: 408.394.3613\r\nFax: 408.562.8100\r\ndfox@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nMiyuki Goldman\r\nQA Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7979\r\nFax: 408.562.8100\r\nmiyuki.goldman@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\nRishi\r\n________________________________________________\r\nRishi Manocha\r\nApplications Engineer\r\nEmail : rmanocha@avocadoit.com\r\nPhone: 408-562-8112\r\nAre you ready for the wireless web?\r\nGet ready by going to http://www.avocadoit.com/",
    "\r\n*****************************\r\nMichael Hughes\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n408-857-8533 mobile\r\n408-562-7961 office\r\n<mail<http://www.avocadoit.com/>",
    "\r\n--\r\nAvocadoIT Europe GmbH\r\nHamborner Strasse 53\r\n40472 Duesseldorf, Germany\r\nphone\t+49 (0) 211.94098-187\r\nfax\t+49 (0) 211.94098-200\r\ncellular\t+49 (0) 171.3845255\r\nmailto:peter.federer@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\nChris\r\n*******************************************************************\r\nChristopher Quinter\r\nDirector of Sales\r\n431 Trumbull Court\r\nNewtown, PA  18940\r\n(267) 757-0736  Mail\r\n(215) 778-3235  Mobile\r\n(413) 714-7699  Fax  www.avocadoit.com\r\n*******************************************************************\r\n",
    "\r\nBest regards,\r\nMeshele Ko\r\nDirector of Sales\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nDirect: 408.562.8034  \tMobile: 408.836.9665 \r\nMain: 408.562.8000 \tFax: 408.562.8100\r\nmko@avocadoit.com \r\nhttp://www.avocadoit.com",
    "\r\n--> Andy\r\n_____________\r\nAndy Wong\r\nDirector, Product Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8131\r\nFax: 408.562.8131\r\nawong@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nMarcia Kadanoff\r\nChief Marketing Officer\r\nAvocadoIT, Inc.\r\n408.562.8050 direct\r\n650.270.4309 mobile\r\nAre you ready for the wireless web?",
    "\r\nJaime L. McNeely \r\neMarketing Manager\r\nAvocadoIT, Inc. \r\n2211 North First Street ~ Suite 200 • San Jose, CA  95131 \r\n408.562.8015 ~ phone •  408.891.8144 ~ cell\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nLaura\r\nLaura C. Henderson\r\nAccount Development Representative\r\nAvocadoIT, Inc.\r\nemail: laura.henderson@avocadoit.com\r\n(800) 355.1103  ext 8187\r\nDirect: (408) 562.8187\r\nhttp://www.avocadoit.com",
    "\r\nRegards,  \r\nJeff Newman\r\nDirector of Sales, Northeast\r\nAvocadoIT, Inc.\r\n508-893-6095  Office\r\n508-740-9126  Mobile\r\n646-304-6312  Fax\r\njeff.newman@avocadoit.com\r\nwww.avocadoit.com",
    "Craig Stewart\r\nVice President\r\nGlobal Telecom Business Unit\r\nOffice: 408-562-8060\r\nFax: 408-562-8100\r\nwww.avocadoit.com",
    "Your Business  Everywhere \r\n*********************************************** \r\nRon Silverton \r\nDirector of Strategic Marketing \r\nAvocadoIT, Inc \r\n2211 N First Street, Suite 200 \r\nSan Jose CA, 95131 \r\nw 408.562-8039 \r\nm 415.845-6789 \r\nf 408.5622-8100 \r\nwww.avocadoit.com \r\n*************************************************",
    '\r\namitabh\r\n_____________________________________________\r\namitabh b. sinha\r\n3231 scott blvd., santa clara, ca  95054\r\n(408)-588-4057 ext. 12 /  (408)-718-2469',
    "\r\n---------------------------------------------------------------------------\r\namitabh b. sinha\r\n2403 s. pond/urbana/il/61801\r\n(217)-344-7256",
    "\r\namitabh\r\n________________________________________\r\namitabh b. sinha\r\n3231 scott blvd, santa  clara, ca 95054\r\n(408) 562-8003  (office)\r\n(408) 718-2469 (cell)",
    "\r\n>---------------------------------------------------------------------\r\n------\r\n>amitabh b. sinha\r\n>2403 s. pond/urbana/il/61801\r\n>(217)-344-7256\r\n>\r\n><< language.zip >>\r\n_______________________________________________________________\r\nGet Free Email and Do More On The Web. Visit http://www.msn.com\r\n",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com <mailto:mkadanoff@iq.com> \r\nwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin\r\n",
    "\r\n----------------------------------------------------------------------\r\nMarcos Sanchez\r\nDir. of Brand Marketing/Minister of Buzz iQ.COM \r\nmsanchez@iq.com\r\n408-777-4000 Fax: 408-777-9444\r\nwww.iq.com\r\niQ.COM Where insiteful promotions begin . . .",
    "\r\n----------------------------------------------------------------------\r\nMarcos Sanchez\r\nDir. of Brand Marketing/Minister of Buzz iQ.COM \r\nmsanchez@iq.com or text page me at 6966952@skytel.com\r\n408-777-4000 Fax: 408-777-9444\r\nwww.iq.com\r\niQ.COM Where insiteful promotions begin . . ."
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 408.777.4040 fax\r\nmkadanoff@iq.com <mailto:mkadanoff@iq.com> \r\nwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin\r\n",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff\r\nChief Marketing Officer\r\niQ.COM\r\n408.861.9242\r\nwww.iq.com <http://www.iq.com/> \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\nNote new direct dial number!\r\n408.861.9242 (talk)\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff\r\nVP & Chief Marketing Officer\r\niQ.COM\r\n408.777.4000 ext. 242 talk\r\n408.777.9444 fax\r\nwww.iq.com\r\nWhere insiteful promotions begin",
    "\r\nVicki Ellis\r\nProgram Manager\r\niQ.COM\r\n408.777.4000 ext. 217\r\nwww.iq.com \r\nvellis@iq.com \r\n'Where insiteful promotions begin'",
    "\r\n----------------------------------------------------------------------\r\nMarcos Sanchez\r\nDir. of Brand Marketing/Minister of Buzz iQ.COM \r\nmsanchez@iq.com or text page me at 6966952@skytel.com\r\n408-777-4000 Fax: 408-777-9444\r\nwww.iq.com\r\niQ.COM Where insiteful promotions begin . . .",
    "\r\nHope all is well with you,\r\nBest regards,Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com\r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\n408.777.4000 ext. 242 talk\r\nmkadanoff@iq.com\r\nwww.iq.com\r\nWhere insiteful promotions begin\r\n",
    "\r\n> \r\n> Marcia Kadanoff\r\n> Chief Marketing Officer\r\n> 408.861.9242 direct\r\n> 408.777.9444 fax\r\n> \r\n> www.iq.com\r\n> Where insiteful promotions begin\r\n> \r\n> \r\n> > ",
    "\r\n> \r\n> MK\r\n> \r\n> Marcia Kadanoff\r\n> VP & Chief Marketing Officer\r\n> 408.777.4000 ext. 242 talk\r\n> \r\n> mkadanoff@iq.com <mailto:mkadanoff@iq.com>\r\n> www.iq.com <http://www.iq.com>\r\n> \r\n> Where insiteful promotions begin\r\n> \r\n> \r\n> \r\n> ",
    "\r\n> \r\n> Marcia Kadanoff\r\n> Chief Marketing Officer\r\n> 408.861.9242 direct\r\n> 408.777.9444 fax\r\n> \r\n> www.iq.com\r\n> Where insiteful promotions begin\r\n> \r\n> > ",
    "\r\nMK\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\nWe\'ve moved!\xa0 Note new phone number: \r\n408.777.4000 ext. 242 talk \r\n408.777.9444 fax \r\nwww.iq.com \r\nmkadanoff@iq.com \r\nWhere insiteful promotions begin",
    "\xa0\r\n\xa0\r\nMK\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\nWe've moved!\xa0 Note new phone number: \r\n408.777.4000 ext. 242 talk \r\n408.777.9444 fax \r\nwww.iq.com \r\nmkadanoff@iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk\xa0\xa0\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMK\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\nWe've moved!\xa0 Note new phone number: \r\n408.777.4000 ext. 242 talk \r\n408.777.9444 fax \r\nwww.iq.com \r\nmkadanoff@iq.com \r\nWhere insiteful promotions begin \r\nIn an emergency - page me at mkadanoff.pager@iq.com",
    "\r\nMK\r\nMarcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin\r\nIn an emergency - page me at mkadanoff.pager@iq.com\r\n>",
    "\r\n> \r\n>MK\r\n>\r\n>Marcia Kadanoff \r\n>Chief Marketing Officer \r\n>408.861.9242 direct \r\n>408.777.9444 fax \r\n>\r\n>www.iq.com \r\n>Where insiteful promotions begin \r\n>\r\n> \r\n>",
    "Marcia Kadanoff\r\nVP & Chief Marketing Officer\r\nWe've moved!  Note new phone number:\r\n408.777.4000 ext. 242 talk\r\nwww.iq.com\r\nmkadanoff@iq.com\r\nWhere insiteful promotions begin\r\nIn an emergency - page me at mkadanoff.pager@iq.com",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk                408.777.4040 fax\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com> \r\nWhere insiteful promotions begin",
    "Marcia Kadanoff \r\nChief Marketing Officer \r\n408.861.9242 direct \r\n408.777.9444 fax \r\nwww.iq.com \r\nWhere insiteful promotions begin",
    "----------------------------------------------------------------------\r\nMarcos Sanchez\r\nDir. of Brand Marketing/Minister of Buzz iQ.COM \r\nmsanchez@iq.com\r\n408-777-4000 Fax: 408-777-9444\r\nwww.iq.com\r\niQ.COM Where insiteful promotions begin . . .",
    "Where insiteful promotions begin\r\n",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk \r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\n\xa0\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\nWe've moved!\xa0 Note new phone number: \r\n408.777.4000 ext. 242 talk \r\n408.777.9444 fax \r\nwww.iq.com \r\nmkadanoff@iq.com \r\nWhere insiteful promotions begin",
    "\r\nMarcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk                408.777.4040 fax\r\nmkadanoff@iq.com <mailto:mkadanoff@iq.com> \r\nwww.iq.com <http://www.iq.com>  \r\n",
    "Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242\xa0talk\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 408.777.4040 fax\r\nmkadanoff@iq.com <mailwww.iq.com <http://www.iq.com>  \r\nWhere insiteful promotions begin",
    "\r\n\xa0Marcia Kadanoff \r\nVP & Chief Marketing Officer \r\n408.777.4000 ext. 242 talk \r\nmkadanoff@iq.com \r\nwww.iq.com \r\nWhere insiteful promotions begin\xa0\r\n",
    "\r\nMelissa A. DePalma\r\n617.679.6478\r\nMelissa_DePalma@Biogen.com\r\n",
    "\r\nVicki Ellis\r\nProgram Manager\r\niQ.COM\r\n408.777.4000 ext. 217\r\nwww.iq.com\r\nvellis@iq.com\r\n'Where insiteful promotions begin'",
    "Melissa A. DePalma\r\n617.679.6478\r\nMelissa_DePalma@Biogen.com",
        "\r\nKind regards\r\nEmma\r\nEmma Lunny\r\nFinancial Services Alliance Manager\r\nHewlett Packard Limited\r\nT: 01344 365840\r\nemma_lunny@hp.com \r\n",
    "\r\nEmma\r\nEmma Lunny\r\nFinancial Services Alliance Manager\r\nHewlett Packard Limited\r\nT: 01344 365840\r\nemma_lunny@hp.com \r\n",
    "\r\nJulie\r\nJulie (Rockwell) Watson\r\nt.(408) 447-5913\r\nf.(408) 447-0003\r\njulie_watson@hp.com\r\nPartner Manager, Emerging Markets\r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n",
    "\r\nJulie\r\nJulie (Rockwell) Watson\r\nt.(408) 447-5913\r\nf.(408) 447-1917\r\njulie_watson@hp.com\r\nMobile Alliances \r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n",
    "\r\nJulie (Rockwell) Watson\r\nt.(408) 447-5913\r\nf.(408) 447-0003\r\njulie_watson@hp.com\r\nPartner Manager, Emerging Markets\r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n",
    "\r\nPatrick Farrow\r\nDesktop & Mobile Computing Specialist\r\nHewlett-Packard Company\r\n203-325-5716\r\npatrick_farrow@hp.com \r\n",
    "\r\nRegards,\r\nJim Lucas\r\nChevron Client Business Manager\r\nHewlett-Packard Company\r\n4430 Willow Rd\r\nSuite 100\r\nPleasanton, CA 94588\r\n925 460 1548 Phone\r\njim_lucas@hp.com\r\n",
    "\r\nThanks,\r\n_____  \r\nvickie hall \r\nmarketing manager, americas &\r\nww internet editorial director\r\nhp mobile e-services bazaar\r\nHewlett-Packard Company\r\n321 Evelyn Road - First Floor\r\nMountain View, CA 94041\r\nwww.hpbazaar.com\r\n650.694.3511 Phone/Mobile\r\n650.694.2081 Fax\r\nvickie_hall@hp.com \r\n_____\r\n",
    "\r\nRegards,\r\n_____  \r\n<http://www.hp.com> \r\nBrian S. Saul\r\nAccount Executive\r\nHewlett-Packard Company\r\n404.648.4040 Phone\r\n770.442.3427 Fax\r\n678.522.4946 Cellular\r\nbrian_saul@hp.com \r\n_____  \r\n",
    "\r\nThanks,\r\nHelen Lum\r\nNorth America Region\r\nHP Complementary Products\r\n***********/-/\\*********************************************************\r\n__       / /  \\    Phone:      650/919-8038  \r\n/_/\\  __ /_/ /\\ \\   Fax:        650/919-8262\r\n\\ \\ \\/ /\\\\ \\ \\_\\ \\  Internet :  Helen_Lum@hp.com\r\n\\ \\ \\/  \\\\ \\ \\/ /  \r\n\\ \\ \\/\\ \\\\ \\  /                                       \r\n\\ \\ \\ \\ \\\\ \\ \\   Address  :  Hewlett-Packard Company\r\n\\ \\ \\ \\ \\\\ \\ \\              690 E. Middlefield Road\r\n\\ \\ \\_\\/ \\ \\ \\             MountainView, CA\r\n\\ \\ \\    \\_\\/             94043\r\n*******\\_\\/*************************************************************  \r\n",
    "\r\nJulie (Rockwell) Watson\r\nt.(408) 447-5913\r\nf.(408) 447-1917\r\njulie_watson@hp.com\r\nMobile Alliances \r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n",
    "\r\nSincerely,\r\nCindy Wu\r\nWorldwide Program Manager\r\nThe Garage Program\r\nWorldwide Solutions Marketing\r\nHewlett-Packard Company\r\n408-447-6875 \r\n",
    "\r\n-- Chan\r\nChanning Benson\r\nchanning_benson@hp.com\r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n(610) 408-6381\r\n>",
    "\r\nCheers\r\nAmit Khanduja\r\nHewlett-Packard Company\r\n2001 Butterfield Road\r\nDowners Grove\r\nIL 60515\r\nph: 630-724-5258\r\n",
    "\r\nregards, Pickett\r\nMobile/Wireless Business Development Manager\r\nHewlett-Packard Company\r\n408-447-1238\r\n",
    "\r\nTalk to you then,\r\n-- Chan\r\nChanning Benson\r\nchanning_benson@hp.com\r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n(610) 408-6381\r\n",
    "\r\nThanks,\r\nJulie\r\nJulie (Rockwell) Watson\r\nt.(408) 447-5913\r\nf.(408) 447-1917\r\njulie_watson@hp.com\r\nMobile Alliances\r\nE-Services Partner Division\r\nHewlett-Packard Company\r\n",
    "\r\nJenifer Heimbach\r\nField Marketing Specialist - EAST\r\nLarge Segment Marketing Team\r\n462-7638\r\n",
    "\r\n--------------------------\r\nFortunata Hermoso\r\nAvocadoIT, Inc.\r\nGlobal Alliance Manager\r\nMobile Phone: 408.464.8998\r\nemail: fortunata.hermoso@avocadoit.com",
    "\r\nMatt Moore\r\nSales Operations Specialist\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nDirect 408.562.8177  Fax 408.562.8101\r\nCell 408.857.8527\r\nMatt.Moore@avocadoit.com",
    "\r\nMarie\r\nMarie B. Washington\r\nVP Finance & Treasurer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\n408-562-7965 - direct\r\n408-562-8100 - fax\r\nmarie.washington@avocadoit.com",
    "\r\nDavid Chan\r\nVP Business Development\r\nAvocadoIT\r\n408-562-8080",
    "\r\nToni Oliveria\r\nSr. Program Manager\r\n2211 North First Street\r\nSuite #200\r\nSan Jose, CA 95131\r\nDirect: 408 562-8183\r\nMobile: 408-390-8148  \r\nFax: 408-562-8100 \r\ntoni.oliveria@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\r\nAndrew Wong\t\t\t\t\tawong@avocadoit.com\r\nDirector of Business Development\t\t408.562.8131 office\r\nAvocadoIT, Inc. \t\t\t\t\t408.562.8100 fax\r\nwww.avocadoit.com\t\t\t\t3231 Scott Blvd, Santa Clara, CA 95054",
    "\r\nRon\r\nEnabling Infrastructure for Post-PC Era\r\n",
    "\r\nEnabling Infrastructure for Post-PC Era\r\n",
    "Enabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************\r\n",
    "Enabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax:  408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************",
    "Regards\r\n>\r\n> Ron\r\n>\r\n> Enabling Infrastructure for Post-PC Era\r\n> ************************************\r\n> Ron D Silverton\r\n> Director of Strategic Marketing\r\n> AvocadoIT, Inc.\r\n> 2211 North First Street\r\n> Suite 200\r\n> San Jose, CA 95131\r\n> Tel: 408.562.8039\r\n> Mobile: 415.845.6789\r\n> Fax:  408.562.8100\r\n> rsilverton@avocadoit.com\r\n> ************************************\r\n>\r\n> ",
    "Enabling Infrastructure for Post-PC Era\r\n************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel: (408) 562-8039\r\nMobile: (415) 845-6789\r\nFax: (408) 408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************",
    "Enabling Infrastructure for Post-PC Era\r\n",
    "\r\n>\r\n>>************************************\r\n>\r\n>Ron D Silverton\r\n>Director of Strategic Marketing\r\n>3231 Scott Boulevard\r\n>Santa Clara, CA  95054\r\n>Tel: (408) 562-8039\r\n>Mobile: (415) 845-6789\r\n>Fax: (408) 408.562.8100\r\n>rsilverton@avocadoit.com\r\n>************************************\r\n>\r\n>\r\n>",
    "\r\n>\r\n> > ************************************\r\n>\r\n> Ron D Silverton\r\n> Director of Strategic Marketing\r\n> AvocadoIT, Inc.\r\n> 3231 Scott Boulevard\r\n> Santa Clara, CA  95054\r\n> Tel: 408.562.8039\r\n> Mobile: 415.845.6789\r\n> Fax:  408.562.8100\r\n> rsilverton@avocadoit.com\r\n> ************************************\r\n>",
    "\r\nDavid\r\nDavid Marcus\r\nSenior Corporate Counsel\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nfax (408) 562-8100",
    "\r\nDavid Marcus\r\nSenior Corporate Counsel\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nfax (408) 562-8100",
    "\r\nDavid Marcus\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDavid Marcus\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nfax (408) 562-8100",
    "\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nMeshele Ko\r\nDirector of Sales\r\nTel: 408.562.8034\r\nCell: 408.836.9665\r\nEmail: mko@avocadoit.com\r\nURL: http://www.avocadoit.com",
    "\r\nMeshele Ko\r\nDirector of Sales\r\n<<...>> \r\nTel: 408.562.8034\r\nCell: 408.836.9665\r\nEmail: mko@avocadoit.com\r\nURL: http://www.avocadoit.com\r\n",
    "\r\nMeshele Ko\r\nDirector of Sales\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nDirect: 408.562.8034  \tMobile: 408.836.9665 \r\nMain: 408.562.8000 \tFax: 408.562.8100\r\nmko@avocadoit.com \r\nhttp://www.avocadoit.com",
    "\r\nBest regards,\r\nMeshele Ko\r\nDirector of Sales\r\nTel: 408.562.4034\r\nCell: 408.836.9665\r\nEmail: mko@avocadoit.com\r\nURL: http://www.avocadoit.com",
    "\r\nMeshele Ko\r\nDirector of Sales\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8034  \tMobile: 408.836.9665 \r\nMain: 408.562.8000 \tFax: 408.562.8100\r\nmko@avocadoit.com \r\nhttp://www.avocadoit.com",
    "\r\nRegards,  bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA  95131\r\nTelephone: 408.562.7989\r\nFax: 408.562.8100\r\nAre you ready for the wireless web? \r\nhttp://www.avocadoit.com",
    "\r\nBest regards,\r\nMeshele Ko\r\nDirector of Sales\r\n<<...>> \r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nDirect: 408.562.8034    Mobile: 408.836.9665 \r\nMain: 408.562.8000      Fax: 408.562.8100\r\nmko@avocadoit.com \r\nhttp://www.avocadoit.com",
    "\r\nDavid Fortney\r\nChief Technology Officer\r\nPaytrust.com\r\nPay your bills in nanoseconds\r\ntel (609) 720-1818  ext. 1773\r\nmob (610) 256-6333\r\ndfortney@paytrust.com\r\n",
    "\r\nMeshele Ko\r\nDirector of Sales\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8034     Mobile: 408.836.9665\r\nMain: 408.562.8000  Fax: 408.562.8100\r\nmko@avocadoit.com\r\nhttp://www.avocadoit.com\r\n",
    "Mike Richman\r\nSenior Account Executive\r\n2211 North first Street, Suite 200\r\nSan Jose, CA  95131\r\n415-460-0644 office\r\n408-857-8732 cell\r\n415-460-0722 fax",
    "\r\nJorg\r\nJorg Heinemann\r\nPartner, Accenture - Electronics & High Tech\r\n1661 Page Mill Road, Palo Alto CA 94304\r\n(650) 213-2047\r\n(650) 465-0222 mobile",
    "\r\nJeff Newman\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n508-893-6095\r\njnewman@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRegards.\r\nJeff Newman\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n508-893-6095\r\njnewman@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nThanks\r\nJeff Newman",
    "\r\nThanks\r\nJeff\r\n--------",
    "\r\nThank you,\r\nJeff Newman",
    "\r\nThank you.\r\nRegards,\r\nJennifer Park\r\nAccount Executive\r\nOffice: 408.562.8185\r\nFax: 408.562.8102\r\nJennifer.Park@AvocadoIT.com\r\nvisit: www.AvocadoIT.com",
    "\r\nRegards,\r\nDavid\r\nDavid Margulies\r\nAvocadoIT Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8100 fax\r\n(650) 520-6575 mobile\r\ndavidm@avocadoit.com\r\n",
    "\r\nDavid Margulies\r\nAdministrative Services Manager\r\nAvocadoIT, Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8100 fax\r\n(408) 891-8145 mobile\r\ndavidm@avocadoit.com",
    "\r\nDavid Margulies\r\nAdministrative Services Manager\r\nAvocadoIT Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8100 fax\r\n(650) 520-6575 mobile\r\ndavidm@avocadoit.com",
    "\r\nDavid Margulies\r\nAvocadoIT Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8100 fax\r\n(650) 520-6575 mobile\r\ndavidm@avocadoit.com",
    "\r\nTalk to you soon,\r\nDavid\r\nDavid Margulies\r\nAdministrative Services Manager\r\nAvocadoIT, Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8100 fax\r\n(408) 891-8145 mobile\r\ndavid.margulies@avocadoit.com",
    "\r\nDavid \r\nDavid Margulies\r\nAvocadoIT Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8000 corporate\r\n(408) 562-8100 fax\r\ndavidm@avocadoit.com",
    "\r\nVice President, Business Development - Asia Pacific\r\nAvocadoIT Inc.",
    "\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: (408) 562-8133 -- Cell: (408) 205-7537\r\nfernand@avocadoit.com",
    "\r\nRepresentative Director\r\nAvocadoIT Japan KK\r\ntel: 650-740-9792 (cell/US), +81-90-4836-7219 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nwww.avocadoit.com",
    "\r\nAshish Grover\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043\r\nwww.avocadoit.com",
    "\r\nRegards,\r\nPrakash Iyer\r\nFounder, CTO &SVP Engineering\r\nAvocadoIT Inc\r\n2211, N. First Street #200\r\nSan Jose , CA 95131\r\n408-562-8030\r\n408-206-1058(cell)",
    "\r\ncheers\r\nAshish\r\nAvocadoIT Inc.\r\nPh. 408-562-8043",
    "prakash Sonie\r\nAvocadoIT Inc\r\n2211, North First Street,\r\nSuite 200,\r\nSan Jose, CA - 95131\r\nPhone: (408) 562 8127\r\nFax:     (408) 562 8100",
    "\r\nPrakash Iyer\r\nFounder & CTO\r\nAvocadoIT Inc\r\n408 206 1058",
    "\r\nCarlos Restrepo\r\nApplications Engineer\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-857-8729\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nSMS: 4083909518@messaging.sprintpcs.com\r\nwww.avocadoit.com",
    "\r\n- Thanks\r\nMadhu Prakash. A\r\nProject Leader\r\nAvocadoIT Inc.\r\n3rd Floor, Auto Plaza,\r\nRoad No: 3, Banjara Hills,\r\nHyderabad - 500 034\r\nMobile\t:091-98480-34501\r\nPh\t:091- 40 - 3354729 \r\n091- 40 - 3357869\r\n091- 40 - 3357870\r\nURL\t: http://www.avocadoit.com\r\nwebmail: madhu_prakash@yahoo.com\r\nmadhu_prakash@hotmail.com",
    "\r\nCarlos Restrepo\r\nApplications Engineer\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-857-8729\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n--\r\nServer Engineering\r\nAvocadoIT Inc.\r\n3231 Scott Blvd.\r\nSantaclara CA 95054",
    "\r\nThanks\r\nMadhu Prakash. A\r\nAvocadoIT Inc. R&D\r\nMobile:091 98480 34501\r\nOffice:\t091 40 3357869\r\n091 40 3357870",
    "\r\nCarlos Restrepo\r\nApplications Engineer\r\nAvocadoIT Inc.\r\n2211 N 1St.\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-390-9518\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nSMS: 4083909518@messaging.sprintpcs.com",
    "\r\nThanks,\r\nFernand Braganza\r\nAvocadoIT Inc.\r\nDirect: 408-562-8133\r\nCell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nthank you\r\nJarmen Givargis\r\nClient Response Rep.\r\n408.562.8195\r\nAvocadoIT Inc.\r\n2211 North First Street suite 200\r\nSan Jose, CA, 95131'",
    "\r\nthank you\r\nJarmen Givargis\r\nClient Response Rep.\r\n408.562.8195\r\nAvocadoIT Inc.\r\n2211 North First Street suite 200\r\nSan Jose, CA, 95131",
    "\r\nThanks,\r\nSagar \r\nDharma Sagar Pathaneni \r\nSr.Software Engineer, \r\nAvocadoIT Inc, \r\n3rd floor,AutoPlaza, \r\nRoad No3 Banjarahills, \r\nHyderabad. \r\nPhone:3357870 Ext 27",
    "\r\nThanks, Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail",
    "\r\n>\r\n> Ashish\r\n> AvocadoIT Inc.,\r\n> Software Er.,\r\n> Ph. 408-562-8043\r\n>\r\n> AvocadoIT Inc.,\r\n> Software Er.,\r\n> Ph. 408-562-8043\r\n>\r\n> ",
    "Sonie\r\nAvocadoIT Inc\r\n3231, Scott Blvd,\r\nSanta Clara, CA - 95054\r\nPhone: (408) 562 8127\r\nFax:     (408) 562 8100",
    "\r\nThanks\r\nAshish Grover\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043",
    "\r\nSincerely, Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel.:   (408) 562-8028\r\nFAX:  (408) 562-8100\r\njschemena@avocadoit.com <mail",
    "\r\nSincerely, Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail",
    "AvocadoIT, Your Business Everywhere\r\n============================================\r\nDavid M. Hosch,  Director of Sales, Canada\r\nAvocadoIT Inc.\r\n49 Ontario Street, 7th Floor\r\nToronto, Ontario,Canada, M5A  2V1\r\nTel: 416-643-4829, Cell: 416-460-3557\r\nFax:416-214-1235\r\nEmail: david.hosch@avocadoit.com\r\nBlackberry: david.hosch@mobile.rogers.com",
    "\r\nThanks, Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel.:   (408) 562-8028\r\nFAX:  (408) 562-8100\r\njschemena@avocadoit.com <mail",
    "\r\n--\r\nNihar Mehta\r\nAvocadoIT Inc. R&D\r\nnihar.mehta@avocadoit.com\r\n(408) 562 8019",
    "\r\nThanks,\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: (408) 562-8133 -- Cell: (408) 205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nAshish Grover\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043",
    "\r\nNihar Mehta\r\nAvocadoIT Inc. R&D\r\nnihar.mehta@avocadoit.com\r\n(408) 562 8019",
    "\r\nCarlos Restrepo\r\nApplications Engineer\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-390-9518\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nSMS: 4083909518@messaging.sprintpcs.com\r\nwww.avocadoit.com",
    "\r\nSincerely, Schemena\r\nSenior Manager\r\nApplications Engineering\r\nAvocadoIT Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel.:   (408) 562-8028\r\nFAX:  (408) 562-8100\r\njschemena@avocadoit.com <mail",
    "Sonie\r\nAvocadoIT Inc\r\n2211, North First Street,\r\nSuite 200,\r\nSan Jose, CA - 95131\r\nPhone: (408) 562 8127\r\nFax:     (408) 562 8100",
    " Thanks,\r\nFernand Braganza\r\nIT\r\nAvocadoIT Inc.\r\nDirect: 408-562-8133\r\nCell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nThanks, Schemena\r\nSenior Manager\r\nApplications Engineering\r\nAvocadoIT Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel.:   (408) 562-8028\r\nFAX:  (408) 562-8100\r\njschemena@avocadoit.com <mail",
    "\r\nThanks\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: 408-562-8133 -- Cell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nFernand Braganza\r\nOperations Support\r\nAvocadoIT Inc.\r\nDirect: 408-562-8133\r\nCell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nCarlos Restrepo\r\nSales Engineer\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-857-8729\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAlex Chavez\r\nAvocadoIT Inc.\r\n4085628098 (direct)\r\n4085628000 (corporate)\r\n4085628100 (fax)\r\nachavez@avocadoit.com",
    "\r\nCarlos Restrepo\r\nApplications Engineer\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-390-9518\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nSMS: 4083909518@messaging.sprintpcs.com",
    "\r\nThanks, \r\nSrilatha Seethamraju,\r\nApplications Engineer,\r\nAvocadoIT Inc.,\r\n3231 Scott Blvd,\r\nSanta Clara, CA  95054.\r\n*   408.562.8073\r\n* ssrilatha@avocadoit.com",
    "\r\nThanks in advance, Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\nTel.:   (408) 562-8028\r\nFAX:  (408) 562-8100\r\njschemena@avocadoit.com <mail",
    "\r\n>\r\n> thanks\r\n>\r\n> Ashish\r\n> AvocadoIT Inc.,\r\n> Software Er.,\r\n> Ph. 408-562-8043\r\n>\r\n> ",
    "\r\nThanks!\r\nDavid Margulies\r\nAvocadoIT Inc.\r\ndavidm@avocadoit",
    "Thanks!\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: 408-562-8133 -- Cell: 408-205-7537\r\nfernand.braganza@avocadoit.com\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: (408) 562-8133 -- Cell: (408) 205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nRegards, \r\nSiebel Alliance Team \r\nMaster Contract: http://www.siebel.com/alliances/ExistingPartners/Contracts/Base_SW.doc\r\nAdditional Contracts and Information: http://www.siebel.com/alliances/ExistingPartners/Contracts/Contracts.zip\r\n",
    "Thanks!\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: 408-562-8133 -- Cell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nTake care.\r\nAlex Chavez \r\nAvocadoIT Inc. \r\n4085628098 (direct) \r\n4085628000 (corporate) \r\n4085628100 (fax) \r\nachavez@avocadoit.com",
    "\r\nBest Regards,\r\nDavid\r\nAvocadoIT,Your Business Everywhere\r\n============================================\r\nDavid M. Hosch, Director of Sales, Canada\r\nAvocadoIT Inc.\r\n49 Ontario Street, 7th Floor\r\nToronto, Ontario\r\nCanada, M5A 2V1\r\nTel:  416-643-4829, Cell: 416-460-3557\r\nFax:  416-643-4832\r\nEmail: david.hosch@avocadoit.com\r\nBlackberry: david.hosch@mobile.rogers.com\r\n",
    "\r\nDavid\r\nDavid Margulies\r\nAvocadoIT Inc.\r\n(408) 562-8119 direct\r\n(408) 562-8000 corporate\r\n(408) 562-8100 fax\r\ndavidm@avocadoit.com",
    "Indira Albert\r\nSoftware Engineer\r\nAvocadoIT Inc,\r\n3rd Floor Auto Plaza,\r\nRoad No. 3,\r\nBanjara Hills 500060\r\nTel :(040)3357869\r\n(040)3357870\r\nFax :(040)3357871\r\nialbert@avocadoit.com\r\nMobile - 98480 49556",
    "\r\nThanks \r\nMadhu Prakash. A\r\nAvocadoIT Inc. R&D\r\nMobile:091 98480 34501\r\nOffice:\t091 40 3357869\r\n091 40 3357870",
    "\r\nRegards,\r\nRuyben.\r\nPhone: 408-562-7988.AvocadoIT Inc,\r\nSan Jose, CA.",
    "\r\nThanks,\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: 408-562-8133 -- Cell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nCarlos Restrepo \r\nSales Engineer \r\nAvocadoIT Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nOffice:  408-562-8052 \r\nMobile: 408-857-8729 \r\nE-mail: Carlos.Restrepo@avocadoit.com \r\nwww.avocadoit.com ",
    "\r\nAshish\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043",
    "\r\n- Nihar\r\nAvocadoIT Inc.\r\n408-562-8019",
    "\r\nThanks\r\nAshish\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043",
    "\r\nThanks\r\nAshish\r\nAvocadoIT Inc.\r\nPh. 408-562-8043",
    "\r\nAShish\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043\r\n",
    "\r\nAshish\r\nAvocadoIT Inc.,\r\nSoftware Er.,\r\nPh. 408-562-8043",
    "\r\nSrilatha Seethamraju,\r\nApplications Engineer,\r\nAvocadoIT Inc.,\r\n3231 Scott Blvd,\r\nSanta Clara, CA  95054.\r\n*   408.562.8073\r\n* ssrilatha@avocadoit.com",
    "\r\n>\r\n> Ashish\r\n>\r\n> AvocadoIT Inc.,\r\n> Software Er.,\r\n> Ph. 408-562-8043\r\n>\r\n> ",
    "\r\nThanks,\r\nFernand Braganza\r\nIT\r\nAvocadoIT Inc.\r\nDirect: 408-562-8133\r\nCell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\nThanks,\r\nLiz\r\nLiz Saiz-Shapiro\r\nDirector New Global Business\r\nAvocadoIT Inc.\r\n3231 Scott Blvd.\r\nSanta Clara, Ca. 95054\r\ndirect 408-562-8115\r\nfax 408-562-8100 \r\nlshapiro@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n____________________________________\r\nFernand Braganza\r\nIT, AvocadoIT Inc.\r\nOffice: (408) 562-8133 -- Cell: (408) 205-7537\r\nfernand.braganza@avocadoit.com",
    "--------------------------\r\nA Wireless Message from AvocadoIT\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nOffice:  408-562-8052\r\nMobile: 408-857-8729\r\nE-mail: Carlos.Restrepo@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nLiz Saiz-Shapiro\r\nDirector New Global Business\r\nAvocadoIT Inc.\r\n3231 Scott Blvd.\r\nSanta Clara, Ca. 95054\r\ndirect 408-562-8115\r\nfax 408-562-8100 \r\nlshapiro@avocadoit.com\r\nwww.avocadoit.com",
    "thx\r\nT\r\nTamra Van Steenburgh\r\nDirector of Strategic Finance\r\nAvocadoIT Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nMain:  408-562-8000\r\nDirect: 408-562-8087\r\nFax:    408-562-8100\r\ntamra.vansteenburgh@avocadoit.com",
    "\r\nThank you,\r\n--------------------------\r\nR. Scott Isgitt\r\nDirector of Sales\r\nAvocadoIT Inc.\r\n512-514-6424 - Office\r\n512-636-1275 - Mobile\r\nscott.isgitt@avocadoit.com\r\n*********************************\r\nMobilized by AvocadoIT Inc.",
    "\r\n________________________\r\nRuben P. Simpliciano\r\nCandelori Communications\r\ntel: 408.774.3414\r\nfax: 408.774.3419\r\nrsimpliciano@candelori.com\r\n",
    "\r\nCheers Vansdadia\r\nSr. Network Engineer\r\nAvocadoIT Inc.",
    "\r\nregards,\r\nToshi Otani\r\nVice President, Business Development - Asia Pacific \r\nAvocadoIT Inc. \r\nRepresentative Director \r\nAvocadoIT Japan KK \r\ntel: 650-740-9792 (cell/US), +81-90-4836-7219 (cell/Jpn) \r\ne-mail: toshiya.otani@avocadoit.com \r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131 \r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo \r\nKamiyacho Mori Building 14F \r\nwww.avocadoit.com",
    "\r\nMadhu Prakash. A\r\nAvocadoIT Inc. R&D\r\nMobile:091 98480 34501\r\nOffice:\t091 40 3357869\r\n091 40 3357870",
    "\r\nCheers,\r\nD'Arcy Salzmann\r\n--- \r\nD'Arcy Salzmann, Product Manager \r\ndarcy.salzmann@avocadoit.com \r\nAvocadoIT, Inc. \r\n49 Ontario Street, 7th floor, Box 24 \r\nToronto, Ontario, M5V 2V1 \r\nCanada \r\nphone    +1.416.643.3593 Toronto, Ont. \r\nphone    +1.408.562.8172 San Jose, CA. \r\nAvocadoIT, your business everywhere. \r\nClick to add my contact info to your Lotus Notes/Netscape/ACT/Outlook \r\naddress book \r\n<http://my.infotriever.com/darcysalzmann>",
    "\r\nGracias,  \r\n--------------------------  Garcia                         rgarcia@avocadoit.com \r\nSales Engineer                         408-829-4827 m \r\nAvocadoIT Inc.                          408-562-7984 w \r\n--- \r\nSent from my AvocadoIT mobile device (www.avocadoit.com) \r\n",
    "\r\nSrilatha Seethamraju,\r\nApplications Engineer,\r\nAvocadoIT Inc.,\r\n2211 N.First Street,\r\nSan Jose, CA  95131.\r\n408.562.8073 - Direct Dial\r\n408.562.8000 - Corporate\r\n408.562.8104 - Fax\r\nsrilatha.seethamraju@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n>\r\n>Richy\r\n>--------------------------\r\n>Ricardo Garcia                         rgarcia@avocadoit.com\r\n>Sales Engineer                         408-829-4827 m\r\n>AvocadoIT Inc.                          408-562-7984 w\r\n>---\r\n>Sent from my AvocadoIT mobile device (www.avocadoit.com)",
    "\r\n>--------------------------\r\n>Ricardo Garcia                         rgarcia@avocadoit.com\r\n>Sales Engineer                         408-829-4827 m\r\n>AvocadoIT Inc.                          408-562-7984 w\r\n>---\r\n>Sent from my AvocadoIT mobile device (www.avocadoit.com)",
    "\r\nFernand Braganza\r\nAvocadoIT Inc.\r\nDirect: 408-562-8133\r\nCell: 408-205-7537\r\nfernand.braganza@avocadoit.com",
    "\r\njuujuu\r\nJarmen Givargis\r\n408.562.8195\r\nAvocadoIT Inc.\r\n2211 N 1st St. Ste 200\r\nSan Jose, CA, 95131",
    "\r\nRegards,\r\nRay Chernaskey\t\r\nDistrict Sales Manager - East\r\nAvocadoIT Inc\r\n610-367-7130",
    "Vice President, Business Development - Asia Pacific\r\nAvocadoIT Inc.",
    "\r\nThanks & Regards,\r\nSreenivas Madiraju,\r\nQuality Assurance Engineer,\r\nAvocadoIT Inc,\r\n2211 North First Street,\r\nSuite 200,\r\nSan Jose, CA 95131\r\nMAIN - 408 - 562 - 8000\r\nDIRECT - 408 - 562 - 8088\r\nFAX - 408 - 562 - 8100",
    "\r\nThanks,\r\nSagar. \r\nDharma Sagar Pathaneni\r\nSr.Software Engineer,\r\nAvocadoIT Inc,\r\n3rd floor,AutoPlaza,\r\nRoad No3 Banjarahills,\r\nHyderabad.\r\nPhone:3357870 Ext 27",
    "\r\nDavid\r\nAvocadoIT,Your Business Everywhere\r\n============================================\r\nDavid M. Hosch, Director of Sales, Canada\r\nAvocadoIT Inc.\r\n49 Ontario Street, 7th Floor\r\nToronto, Ontario\r\nCanada, M5A 2V1\r\nTel:  416-643-4829, Cell: 416-460-3557\r\nFax:  416-643-4832\r\nEmail: david.hosch@avocadoit.com\r\nBlackberry: david.hosch@mobile.rogers.com\r\n",
    "\r\nSee you then, Schemena\r\nDirector\r\nApplications Engineering\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mail",
    "\r\nThanks,  \r\nSrilatha Seethamraju,\r\nApplications Engineer,\r\nAvocadoIT Inc.,\r\n2211 N.First Street,\r\nSan Jose, CA  95131.\r\n408.562.8073 - Direct Dial\r\n408.562.8000 - Corporate\r\n408.562.8104 - Fax\r\nsrilatha.seethamraju@avocadoit.com",
        "\r\n_______________________________________________________________________\r\nMurali B. Subbarao      \r\nDirector, Strategy and Alliances    \r\nHP Financial Services Business Unit   \r\nPhone: +1 408 447-6476  Fax: +1 408 447 0571\r\nEmail: Murali_Subbarao@hp.com\r\nURL: www.hp.com/go/fsi\r\n_____________________________________________________________________\r\n",
    "\r\nMurali B. Subbarao\r\nVP Solutions\r\nArula Systems\r\n10670 N. Tantau Ave., Suite 200\r\nCupertino, CA  95014\r\n408-996-3400 ext. 237\r\nFax: 408-996-3406\r\nmurali.subbarao@arula.com\r\nwww.arula.com\r\n",
    "\r\ncheers\r\nAshish Grover\r\nEr. - R&D\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8043\r\nashish.grover@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nthanks\r\nAshish Grover\r\nAvocadoIT, Inc.  ( R&D )\r\n2211 N. First Street, Suite 200, \r\nSan Jose, CA 95131\r\nPhone: 408-562-8043\r\nFax: 408-562-8103\r\nemail: ashish.grover@avocadoit.com\r\nwww.avocadoit.com\r\n( AvocadoIT, Your Business Everywhere! )",
    "\r\nthanks\r\nAshish Grover\r\nEr. - R&D\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8043\r\nashish.grover@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nAshish Grover\r\nEr. - R&D\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8043\r\nashish.grover@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nTony\r\nAnthony N. Tarsia\r\nDirector of Sales\r\nAvocadoIT, Inc.\r\n4094 Majestic Lane, PMB 127\r\nFairfax, VA 22033\r\nOffice - 703-273-6313      Cell - 703-861-5292\r\nFax - 703-995-4722          E-mail - anthony.tarsia@avocadoit.com\r\nhttp://www.avocadoit.com",
    "\r\nNora Nagy\r\nQA Engineer\r\n408.562.8047 Direct\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nnora.nagy@avocadoit.com",
    "\r\nThanks.\r\nEdward Tian\r\nUI team, Engineering\r\nAvocadoIT, Inc.\r\n2211 North 1st Street, suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8122\r\nFax: 408-562-8103\r\netian@avocadoit.com",
    "\r\nBarry Phillips\r\nDirector, Customer Marketing \r\n408.562.8016 Direct\r\n408.562.8100 Fax\r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nbarry.phillips@avocadoit.com",
    "\r\nthanks,\r\nRobert\r\nRobert Moore, Global Alliance Manager\t\t\t\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100\r\nrobert.moore@avocadoit.com",
    "\r\nJuan Carlos Estíbariz -- AvocadoIT Canada -- http://www.avocadoit.com\r\n'",
    "\r\nNora\r\nNora Nagy\r\nQA Engineer\r\n408.562.8047 Direct\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nnora.nagy@avocadoit.com",
    "\r\nNora\r\nNora Nagy\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408.562.8047\r\n408.588.0060 fax\r\nnnagy@avocadoit.com",
    "\r\nNora Nagy\r\n3231 Scott Boulevard\r\nSanta Clara, CA  95054\r\n408.562.8047\r\n408.588.0060 fax\r\nnnagy@avocadoit.com",
    "\r\nThanks,\r\nNora \r\nNora Nagy\r\nQuality Assurance\r\n408.562.8047 Direct\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nnora.nagy@avocadoit.com",
    "\r\nThanks,\r\nNora\r\nNora Nagy \r\nQA Engineer\r\n408.562.8047 Direct",
    "\r\nBarry\r\nBarry Phillips\r\nDirector, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-8016 (direct)\r\n(408) 562-8100 (fax)\r\nwww.avocadoit.com",
    "\r\nBarry Phillips\r\nDirector, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-8016 (direct)\r\n(408) 562-8100 (fax)\r\nwww.avocadoit.com",
    "\r\nPhilip O\'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 212-666-1194\r\nhttp://www.avocadoit.com",
    "\r\nThanks\r\nPhilip O'Connell \r\nDirector of Sales \r\nAvocadoIT \r\nTelephone 212-749-0852 \r\nFax 309-416-3937 \r\nhttp://www.avocadoit.com",
    "\r\nPhilip O'Connell (NJ & NY)\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\nRegards,\r\nPhilip O'Connell \r\nDirector of Sales \r\nAvocadoIT \r\nTelephone 212-749-0852 \r\nFax 309-416-3937 \r\nhttp://www.avocadoit.com",
    "\r\n-- Philip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\nThanks --\r\nPhilip O'Connell\r\nDirector of Sales\r\nAvocadoIT\r\n187 Claremont Ave\r\nNY 10027\r\nTelephone 212-749-0852\r\nFax 309-416-3937\r\nhttp://www.avocadoit.com",
    "\r\nJuan Carlos Estíbariz -- AvocadoIT Canada -- http://www.avocadoit.com\r\n",
    "\r\nMarilisa Walski\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n408.562.8144  408.562.8100 fax\r\nmarilisa.walski@avocadoit.com\r\nwww.avocadoit.com",
    "Marilisa Walski\r\niQ.COM\r\nMarketing Coordinator\r\n408.777.4000 Ext. 223\r\n408.777.9444 Fax\r\nmwalski@iq.com\r\nwww.iq.com\r\nWhere Insiteful Promotions Begin.",
    "\r\n++++++++++++++++++++++++++++++++++++++++++++++++\r\nRoy Chao\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  \xa0PH:(408)861-9218\r\nRegional Sales Manager\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0FX:(408)777-9444\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\r\niQ.COM\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0\r\n12950 Saratoga Ave., Suite B\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0rchao@iq.com\r\nSaratoga, CA\xa0 95070\r\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Where Insiteful Promotions Begin\r\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0http://www.iq.com\r\n++++++++++++++++++++++++++++++++++++++++++++++++",
    "\r\n++++++++++++++++++++++++++++++++++++++++++++++++\r\nRoy Chao\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  \xa0PH:(408)777-4000\r\nRegional Sales Manager\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0ext. 218\r\niQ.COM\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 \xa0FX:(408)777-9444\r\n12950 Saratoga Ave., Suite B\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0rchao@iq.com\r\nSaratoga, CA\xa0 95070\r\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Where Insiteful Promotions Begin\r\n\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0http://www.iq.com\r\n++++++++++++++++++++++++++++++++++++++++++++++++",
    "\r\nMarilisa Walski\r\nMarketing Coordinator\r\n408.777.4000 Ext. 223\r\n408.777.9444 Fax\r\nmwalski@iq.com\r\nwww.iq.com\r\nWhere Insiteful Promotions Begin.",
    "\r\nThanks,\r\nTim \r\nTim Welch\r\nRegional Sales Manager\r\niQ.com\r\n12950 Saratoga Ave. Suite B\r\nSaratoga, CA  95070\r\ntwelch@iq.com\r\n(408)861-9257\r\n408 867-4366 fax\r\nwww.iq.com\r\nInsiteful promotions",
    "\r\nBarbara Camozzi\r\nDirector, Relationship Marketing\r\niQ.COM -- Where Insiteful Promotions Begin\r\n408.777.4000 ext. 256 talk\r\n408.867.4366 fax",
    "L.\r\nLeslie Latham\r\nVP Sales & Business Development\r\niQ.COM\r\n408 777-4000 x220\r\n408 777-9444 fax\r\n415 560-1475 page\r\nhttp://www.iq.com\r\nmail\r\nInsiteful Promotions",
    "\r\n--\r\nRené Romain\r\nManaging Director\r\nAvocadoIT Europe\r\nLeopoldstr. 236/III\r\n80807 Munich\r\nGermany\r\nWork: +49 (0) 89 35874-310\r\nCellular: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com\r\nWeb : www.avocadoit.com\r\n",
    "\r\nBis bald. René\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice President - AvocadoIT\tEurope\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice President - Business Development\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice President - Business Development \t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "\r\nRegards, René\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVP - Business Development\t\tWork: +49 (0) 89 35874-310\r\nAvocadoIT Europe \t\t\t\tCellular: +49 (0) 170 4493066\r\nLeopoldstr. 236/III \t\t\tFax: +49 (0) 89 35874-610\r\n80807 Munich           \t\t\tE-mail : rromain@avocadoit.com\r\nGermany\t\t\t\t\tWeb : www.avocadoit.com\r\n---------------------------------------------------------------------------",
    "\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice Preseident - Business Development\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice President - Business Development\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------\r\n",
    "\r\nRegards, René\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nVice President - Business Development\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "\r\n--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------\r\n",
    "\r\n------------------------------------------------------------------------\r\n--------\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n------------------------------------------------------------------------\r\n--------\r\n",
    "\r\nCiao, René\r\n----------------------------------------------------------------------------\r\n----\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n----------------------------------------------------------------------------\r\n----\r\n",
    "\r\nR´.\r\n----------------------------------------------------------------------------\r\n----\r\nRené Romain\r\nManaging Director\r\nAvocadoIT Europe\r\nLeopoldstr. 236/III\r\n80807 Munich\r\nGermany\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n----------------------------------------------------------------------------\r\n----\r\n",
    "\r\nRené Romain  \r\n------------------------------------------------------------------------\r\n----\r\n----\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n------------------------------------------------------------------------\r\n----\r\n----\r\n",
    "\r\nRené\r\n----------------------------------------------------------------------------\r\n----\r\nRené Romain\r\nVice President - Business Development\r\nAvocadoIT Europe\r\nLeopoldstr. 236/III\r\n80807 Munich\r\nGermany\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n----------------------------------------------------------------------------\r\n----\r\n",
    "\r\nThks. René\r\n-----------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "\r\n--------------------------------------------------------------------------------\r\nRené Romain \r\nVice President - Business Development\t\t\t\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "--------------------------------------------------------------------------------\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailWeb : www.avocadoit.com\r\n--------------------------------------------------------------------------------",
    "\r\nCiao. R.\r\n----------------------------------------------------------------------------\r\n----\r\nRené Romain \t\t\t\r\nManaging Director\t\t\r\nAvocadoIT Europe \t\t\r\nLeopoldstr. 236/III \t\t\r\n80807 Munich                 \t\r\nGermany\t\t\t\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n----------------------------------------------------------------------------\r\n----\r\n",
    "\r\nAll the best, René\r\nhttp://www.avocadoit.com/press/release9_12_00.html\r\n----------------------------------------------------------------------------\r\n---\r\nRené Romain\r\nManaging Director\r\nAvocadoIT Europe\r\nLeopoldstr. 236/III\r\n80807 Munich\r\nGermany\r\nWork: +49 (0) 89 35874-310\r\nMobile: +49 (0) 170 4493066\r\nFax: +49 (0) 89 35874-610\r\nE-mail : rromain@avocadoit.com <mailto:rromain@avocadoit.com>\r\nWeb : www.avocadoit.com\r\n----------------------------------------------------------------------------\r\n----\r\n",
    "\r\nGiselle Bisson\r\nDirector of Corporate Communications\r\nAvocadoIT\r\nmobile phone NEW NUMBER: 408.910.6598\r\noffice: 408.562.8004\r\ngbisson@avocadoit.com\r\ngisellian@aol.com\r\nhttp://www.avocadoit.com\r\nAre you ready for the wireless web?",
    "Lady\r\nVP, Business Development \r\nAvocadoIT, Inc.\r\n2211 N. 1st Street, San Jose, CA 95131\r\np:  408-562-8070  m:  408-857-8745   f:  408-562-8100  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nHave a great weekend. Lady\r\nVP, Business Development - Content\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  415-505-3756   f:  415-677-9972  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?",
    "Lady\r\nVP, Business Development\r\nAvocadoIT\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  408-562-8070   f:  408-562-8100   m:  415-505-3756\r\nAre you ready for the wireless web?",
    "\r\nHope all is well. Lady\r\nVP, Business Development - Content\r\nAvocadoIT, Inc.\r\n152 Lombard St., Suite 606, San Francisco, CA 94111\r\np:  415-505-3756   f:  415-677-9972  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nGrant Frederiksen\r\nSales Director - Southeast\r\nAvocadoIT\r\n770.753.9837  (o)\r\n770.753.9581  (f)\r\n770.294.6496  (m)\r\ngrantf@avocadoit.com\r\nwww.avocadoit.com\r\nAre you ready for the wireless web?",
    "Lady\r\nVP, Business Development - Content\r\nAvocadoIT\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  408-562-8070   f:  408-677-9972   m:  415-505-3756\r\nAre you ready for the wireless web?",
    "\r\nThanks Lady\r\nVP, Business Development - Content\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  415-505-3756   f:  415-677-9972  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nmk\r\nwww.avocadoit.com <http://www.avocadoit.com> \r\nAre you ready for the wireless web?\r\n",
    #"\r\nAre you ready for the wireless web?\r\n",
    "Lady\r\nVP, Business Development - Content\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  415-505-3756   f:  415-677-9972  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nBen\r\n_________________________ \r\nH. Benjamin Groban \r\nBusiness Development \r\nGoAmerica Communications \r\n433 Hackensack Ave., 3rd fl. \r\nHackensack, NJ 07601 \r\nbgroban@goamerica.net \r\nTel: 201-996-1717 Ext. 2370 \r\nEfax: 603-250-5739 \r\n_________________________ \r\n",
    "Lady\r\nVP, Business Development - Content\r\nAvocadoIT, Inc.\r\n152 Lombard St., Suite 606, San Francisco, CA 94111\r\np:  415-505-3756   f:  415-677-9972  \r\nemail:  trish.lady@avocadoit.com\r\nAre you ready for the wireless web?",
    "\r\nThanks,\r\nRishi\r\n________________________________________________\r\nRishi Manocha\r\nProject Coordinator\r\nEmail : rishi.manocha@avocadoit.com\r\nDirect: 408-562-8112\r\nMobile: 408-857-8746\r\nAre you ready for the wireless web?\r\nGet ready by going to http://www.avocadoit.com/",
    "\r\nRishi\r\n________________________________________________\r\nRishi Manocha\r\nApplications Engineer\r\nEmail : rishi.manocha@avocadoit.com\r\nPhone: 408-562-8112\r\nAre you ready for the wireless web?\r\nGet ready by going to http://www.avocadoit.com/",
    "\r\nAre you ready for the wireless web?\r\n",
    "Trish Lady\r\nVP, Business Development - Content\r\nAvocadoIT\r\n3231 Scott Blvd., Santa Clara, CA 95054\r\np:  408-562-8070   f:  408-562-8100   m:  415-505-3756\r\nAre you ready for the wireless web?",
    "\r\nGiselle Bisson \r\nDirector of Corporate Communications \r\nAvocadoIT \r\nmobile phone NEW NUMBER: 408.910.6598 \r\noffice: 408.562.8004 \r\ngbisson@avocadoit.com \r\ngisellian@aol.com \r\nhttp://www.avocadoit.com \r\nAre you ready for the wireless web?",
    "\r\nAndrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n2211 N. First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\nandrew.wagoner@avocadoit.com",
    "\r\nAndrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n2211 N. First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\nAndrew.Wagoner@every path.com",
    "Andrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n2211 N. First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\nandrew.wagoner@avocadoit.com",
    "Andrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n3231 Scott Blvd\r\nSanta Clara, CA 95054\r\nandrew.wagoner@avocadoit.com",
    "\r\nAndrew Wagoner\r\nPurchasing Agent\r\n408.562.8174 Direct \r\n2211 N. First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\nAndrew.Wagoner@avocadoit.com",
    "\r\nToni Oliveria\r\nSr. Partner Program Manager\r\n2211 North First Street\r\nSuite #200\r\nSan Jose, CA 95131\r\nDirect: 408 562-8183\r\nMobile: 408-390-8148  \r\nFax: 408-562-8100 \r\ntoni.oliveria@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nToni Oliveria\r\nSr. Program Manager\r\n3231 Scott Blvd.\r\nSanta Clara, California  95054\r\nMain: 408 562-8000\r\nDirect: 408 562-8183\r\nMobile: 408-390-8148  \r\nFax: 408-562-8100 \r\ntoni.oliveria@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nMeshele Ko\r\nDirector of Sales\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8034  \tMobile: 408.836.9665 \r\nMain: 408.562.8000 \tFax: 408.562.8100\r\nmeshele.ko@avocadoit.com \r\nhttp://www.avocadoit.com",
    "\r\nMeshele Ko\r\nDirector of Sales\r\nTel: 408.588.4057 ext 42\r\nCell: 408.836.9665\r\nEmail: mko@avocadoit.com\r\nURL: http://www.avocadoit.com",
    "\r\nThanks,\r\nchandu\r\nP. Chandra Sekhar\t\t\t\t\t\t\tOffice: +91 40 335 7870\r\nProject Manager\t\t\t            \t\tMobile: +91 98480 14678 \r\nwww.avocadoit.com",
    "\r\nP. Chandra Sekhar\t\t\t\t\t\tOffice: +91 040 335 7870\r\nProject Manager\t\t\t                          Mobile: +91 98480 14678 \r\nwww.avocadoit.com",
    "\r\nchandra \r\nP. Chandra Sekhar\t\t\t\t\t\t\tOffice: +91 40 335 7870\r\nProject Manager\t\t\t            \t\tMobile: +91 98480 14678 \r\nwww.avocadoit.com",
    "\r\nThanks and regards,\r\nChandra Sekhar \r\n____________________________________________________________________\r\nGet free email and a permanent address at http://www.netaddress.com/?N=1\r\n",
    "\r\nThanks,\r\nchandu.\r\nP. Chandra Sekhar\t\t\t\t\t\t\tOffice: +91 40 335 7870\r\nProject Manager\t\t\t            \t\tMobile: +91 98480 14678 \r\nwww.avocadoit.com",
    "\r\nThanks,\r\nChandu  \r\nP. Chandra Sekhar\t\r\nProject Manager\r\nOffice: +91 40 335 7870\r\nMobile: +91 98480 14678",
    "\r\nthanks.\r\nP. Chandra Sekhar\t\t\t\t\t\t\tOffice: +91 40 335 7870\r\nProject Manager\t\t\t            \t\tMobile: +91 98480 14678 \r\nwww.avocadoit.com",
    "\r\nThanks\r\nchandra sekhar\r\nP. Chandra Sekhar \r\nSr. Software Engineer\r\nAvocadoIT.\r\nHyderabad, India 500 034\r\nOffice: 040-335 7869/ 70  x38\r\nHome: 040- 3042551\r\nMobile: +91 98480 14678\r\ncputigampu@avocadoit.com",
    "\r\nthanks.\r\n<< OLE Object: Picture (Metafile) >> \r\nP. Chandra Sekhar\t\t\t\t\t\t\tOffice: +91 40 335 7870\r\nProject Manager\t\t\t            \t\tMobile: +91 98480 14678 \r\nwww.avocadoit.com\r\n",
    "\r\nbye\r\nChandu\r\n________________________________\r\nChandra Sekhar Putigampu,\r\nReliance Infocomm,\r\nJ3- SB16, 1st Floor, DAKC\r\nThane-Belapur Rd,Navi Mumbai-400709.\r\nPhone: 022-27624000 Ext. 3513\r\n",
    "\r\n-Chandu\r\nP. Chandra Sekhar\t\r\nProject Manager\r\nOffice: +91 40 335 7870\r\nMobile: +91 98480 14678",
    "\r\nThanks\r\nChandu\r\nP. Chandra Sekhar\t\r\nProject Manager\r\nOffice: +91 40 335 7870\r\nMobile: +91 98480 14678",
    "\r\nThanks,\r\nChandu\r\nP. Chandra Sekhar\t\r\nProject Manager\r\nOffice: +91 40 335 7870\r\nMobile: +91 98480 14678",
    "\r\nP. Chandra Sekhar\t\r\nProject Manager\r\nOffice: +91 40 335 7870\r\nMobile: +91 98480 14678",
    "\r\nThanks,\r\nChandu ",
    "\r\nMark Tracy\r\navocadoit.com\r\n3231 Scott Blvd., Santa Clara, CA  95054\r\nPhone: 408.588.4057 Ext. 21 / Fax: 408. 588.0060",
    "\r\nMark Tracy\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200, San Jose, CA  95131\r\nPhone: 408.562.8006 / Fax: 408. 588.0060",
    "\r\nMark Tracy\r\nAvocadoIT, Inc.\r\n3231 Scott Blvd., Santa Clara, CA  95054\r\nPhone: 408.562.8006 / Fax: 408. 588.0060",
    "\r\nMark\r\nMark Tracy\r\navocadoit.com\r\n3231 Scott Blvd., Santa Clara, CA  95054\r\nPhone: 408.562.8006 / Fax: 408. 588.0060",
    "\r\nThanks.\r\nMark Tracy \r\nAvocadoIT, Inc.\r\n3231 Scott Blvd., Santa Clara, CA  95054\r\nPhone: 408.562.8006 / Fax: 408. 588.0060",
    "\r\nMark Tracy\r\navocadoit.com\r\n3231 Scott Blvd., Santa Clara, CA  95054\r\nPhone: 408.562.8006 / Fax: 408. 588.0060",
        "\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: \r\nFax: 408.562.8100 \r\ndoug.field@avocadoit.com \r\nwww.avocadoit.com",
    "Doug Field\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: \r\nFax: 408.562.8100 \r\ndoug.field@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nSr. Instructor \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\nDirect: 408.562.7956 \r\nMobile: 408.857.8744 \r\nFax: 408.562.8100 \r\ndoug.field@avocadoit.com \r\nPalm:  ep036@palm.net \r\nwww.avocadoit.com",
    "\r\nRegards,\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: \r\nFax: 408.562.8100 \r\ndouglas.field@avocadoit.com \r\nwww.avocadoit.com",
    "\r\n________________________  bin Abdulmughnee \r\nSystems Engineer \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nDirect: 408.562.7964 \r\nMobile: 408.206-8122 \r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com",
    "Stewart \r\nVice President \r\nGlobal Telecom Business \r\n____________________________ \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, California 95131 \r\nMobile:   408-891-8139 \r\nOffice:    408-562-8060  \r\nFax:       408-562-8100 \r\nEmail:    craig.stewart@avocadoit.com \r\nWeb:      www.avocadoit.com",

    "-------------------------- bin Abdulmughnee\r\nSystems Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7964\r\nMobile: 408.206-8122\r\nFax: 408.562.8100 \r\nrayhan.abdulmughnee@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nRegards,\r\nVinayak Dhanapune\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8085\r\nFax: 408.562.8100\r\nvinayak.dhanapune@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n> dbm\r\n> \r\n> \r\n> \r\n> Debbie Margulies\r\n> Marketing Manager\r\n> AvocadoIT, Inc.\r\n> 2211 North First Street\r\n> Suite 200\r\n> San Jose, CA 95131\r\n> Main: 408.562.8000\r\n> Direct: 408.562.8059\r\n> Mobile: 408.393.1052\r\n> Fax: 408.562.8100\r\n> debbie.margulies@avocadoit.com\r\n> www.avocadoit.com\r\n> \r\n> AvocadoIT, Your Business Everywhere!\t\r\n>",
    "\r\nBithi\r\nBithi Agrawal\r\nApplications Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8071\r\nFax: 408.562.8100\r\nbagrawal@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThank you,\r\nMatt \r\nMatt Moore\r\nSales Operations Specialist\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nDirect 408.562.8177  Fax 408.562.8101\r\nMatt.Moore@avocadoit.com",
    "\r\nThanks,\r\nMark\r\nMark Stewart \r\nFacility Manager \r\n408.562.8042 Direct & Fax \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nmark.stewart@avocadoit.com ",
    "\r\nMark\r\nMark Stewart\r\nFacility Manager\r\n408.562.8042 Direct & Fax\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nmark.stewart@avocadoit.com ",
    "\r\n- Dave\r\nDave Sulcer\r\nUI Team, Engineering\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\nDirect: 408.562.7959\r\nMobile: 408-768-2601\r\nFax: 408-562-8103\r\nDave.Sulcer@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nJennifer Park\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\ndirect: 408.562.8185\r\nfax: 408.562.8102\r\nwww.avocadoit.com",
    "\r\nMark\r\nMark Stewart\r\nFacility Manager\r\n408.562.8042 Direct & Fax\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nmark.stewart@avocadoit.com",
    "\r\nThanks.\r\nAnn\r\nAnn Sasso\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131 \r\n(408) 562-7973\r\nann.sasso@avocadoit.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks\r\nAlice\r\nAlice Sun\r\nStaff Accountant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8048\r\nEmail: asun@avocadoit.com",
    "\r\n*********************************************\r\nT. Ty Wang\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7945 Office\r\n(408) 562-8100 Fax\r\nty.wang@avocadoit.com\r\n*********************************************",
    "\r\nThanks,\r\nRyan\r\n____________________________\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 office\r\n408.562.8100 fax\r\n408.420.3841 mobile\r\nryan.glover@avocadoit.com\r\nwww.avocadoit.com",
    '\r\nRepresentative Director\r\nAvocadoIT Japan KK\r\ntel: 408-205-9961 (cell/US), +81-90-4836-7219 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nHP: www.avocadoit.com/jp\r\n"AvocadoIT, Your Business Everywhere!" \r\n***********************************************************',
    "\r\nPrakash Iyer\r\nFounder & Chief Technology Officer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8030 \r\nMobile: 408.206.1058\r\nFax: 408.562.8100 \r\nprakash.iyer@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\nMitch Stewart\r\nDirector of Client Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8121 direct  (408) 562-8056 fax",
    "\r\nThanks,\r\nSusan\r\nSusan Simmons\r\nDirector of Finance & Administration\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8022 direct dial\r\n(408) 562-8000 corporate\r\n(408) 562-8100 fax\r\nssimmons@avocadoit.com",
    "\r\nT. Ty Wang \r\nDirector, Market Development \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nDirect: 408.562.7945 \r\nFax: 408.562.8100 \r\nty.wang@avocadoit.com \r\nwww.avocadoit.com ",
    "\r\nKelsey Kerr\r\nAccount Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n408.562.8036 (office)\r\n408.394.1054 (cell)\r\n408.562.8100 (fax)\r\nkkerr@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nRobert \r\nRobert Moore\r\nGlobal Alliance Manager\t\t\t\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100 \r\nrobert.moore@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n__________________________________________________\r\nDavid Swanson\r\nVice President, Human Resources\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8130\r\nMobile: 408.205.9958\r\nFax: 408.562.8106\r\ndavid.swanson@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks\r\nNadir Nawabzada\r\nSystem Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nPhone: 408-562-8044\r\nCell: 408-205-7289\r\nEmail: nnawabzada@avocadoit.com",
    "\r\nThank you.\r\nDoug Adkins\r\nDirector, Technical Support\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, Ca. 95131\r\nOffice:(408) 562-7985\r\nMobile: (408) 829-5118\r\nFax: (408) 562-8100\r\nEmail: doug.adkins@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\nThank you,\r\nRyan\r\n____________________________\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 direct\r\n408.562.8100 fax\r\nryan.glover@avocadoit.com",
    "\r\n-Lisa\r\nLisa Zissen\r\nExec. Asst. to Prakash Iyer, CTO & Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, California  95131\r\n(408) 562-8012 - direct\r\n(408) 562-8104 - Fax\r\nlisa.crowder@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nMark Stewart\r\nFacility Manager\r\n408.562.8042 Direct & Fax\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nmark.stewart@avocadoit.com ",
    "\r\nRegards,\r\nAlex Tran\r\nDirector of Product Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8068 office\r\n(408) 562-8100 fax\r\n(408) 420-3877 cell\r\nalex.tran@avocadoit.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n*New Address*\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks!\r\nDaimyo   \r\nwww.avocadoit.com \r\nDaimyo Chen\r\nSystems Admin\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect:\t (408) 562-7968\r\nMobile:\t (408) 891-8138",
    "\r\nRobert Moore\r\nGlobal Alliance Manager\t\t\t\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8078\r\nMobile: 408.420.3852\r\nFax: 408.562.8100 \r\nrobert.moore@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThank you\r\nPhani Yadavalli\r\nPhani Syam Yadavalli\r\nAvocadoIT,Inc\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8074 (W)\r\n(408) 261-2602 (H)",
    "\r\nThanks,\r\nDaimyo\r\nwww.avocadoit.com \r\nDaimyo Chen\r\nSystems Admin\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect:\t (408) 562-7968\r\nMobile:\t (408) 891-8138",
    "\r\nDavid Chan, VP Business Development\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8080 \r\nMobile: 415.819.5490\r\nFax: 408.562.8100",
    "\r\nThanks,\r\nSusan\r\nSusan Simmons\r\nDirector of Finance & Administration and Corporate Controller\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8022\r\nMobile: 408.857-8738\r\nFax: 408.562.8100\r\nsusan.simmons@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks.\r\nAlex Tran\r\nDirector of Product Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8068 office\r\n(408) 562-8100 fax\r\n(408) 420-3877 cell\r\nalex.tran@avocadoit.com",
    "\r\nMarie B. Washington\r\nVP Finance & Treasurer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\n408-562-7965 - direct\r\n408-562-8100 - fax\r\nmarie.washington@avocadoit.com",
    "\r\nMichelle\r\nMichelle Plesha\r\nAssistant Controller\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect:  408.562.8118\r\nFax:  408.562.8100\r\nmichelle.plesha@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks\r\nMark\r\nMark Stewart \r\nFacility Manager \r\n408.562.8042 Direct & Fax \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nmark.stewart@avocadoit.com ",
    "\r\nDavid Marcus\r\nDirector\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com \r\nwww.avocadoit.com ",
    "\r\nRuth Gantly\r\nManager, Information Products\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nemail ruth.gantly@avocadoit.com\r\nPhone: 408.562.8033\r\nFax: 408.562.8100\r\n",
    "\r\nthanks\r\njaime\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 phone\r\n408.891.8144 cell\r\nwww.avocadoit.com",
    "\r\nThanks!\r\nDaimyo\r\nwww.avocadoit.com \r\nDaimyo Chen\r\nSystems Admin\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect:\t (408) 562-7968\r\nMobile:\t (408) 891-8138",
    "\r\nThanks\r\nNadir\r\nNadir Nawabzada\r\nSystem Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8044\r\nCell: 408-205-7289",
    "\r\nThanks & Regards,\r\nSreenivas Madiraju,\r\nQuality Assurance Engineer,\r\nAVOCADOIT.COM,\r\n2211 North First Street,\r\nSuite 200,\r\nSan Jose, CA 95131\r\nMAIN - 408 - 562 - 8000\r\nDIRECT - 408 - 562 - 8088\r\nFAX - 408 - 562 - 8100",
    "Thank you!\r\nTeresa M. Neves \r\nFinancial Services Coordinator\r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\nDirect: 408.562.8196 \r\nFax: 408.562.8107\r\nteresa.neves@avocadoit.com \r\nwww.avocadoit.com ",
    "\r\nStay in touch,\r\nDonivan\r\nDonivan Fox\r\nManager, Partner Solutions Group\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8001\r\nMobile: 408.857.8534\r\nFax: 408.562.8100\r\ndonivan.fox@avocadoit.com\r\nwww.avocadoit.com",
    "\r\n--------------------------  Garcia \r\nSales Engineer \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n(408) 562-7984 (direct) \r\n(408) 829-4827 (mobile) ",
    "\r\n---  Garcia \r\nTechnical Marketing Engineer \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\n(408) 562-7984 office \r\n(408) 562-8100 fax \r\n(408) 829-4827 cell \r\nricardo.garcia@avocadoit.com \r\nwww.avocadoit.com \r\n",
    "\r\nNadir Nawabzada\r\nSystem Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8044\r\nCell: 408-205-7289",
    "************************************\r\nRon D Silverton\r\nDirector of Strategic Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131\r\nTel: 408.562.8039\r\nMobile: 415.845.6789\r\nFax: 408.562.8100\r\nrsilverton@avocadoit.com\r\n************************************\r\n<> \r\n> ",
    "\r\nRegards,\r\nSteven Hirata\r\nDirector of Networks and Operations\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8065 \r\nFax: 408.562.8100 \r\nshirata@avocadoit.com",
    "\r\nThanks\r\nAlice Sun\r\nStaff Accountant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8048\r\nEmail: asun@avocadoit.com",
    "\r\njaime\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 phone\r\n408.891.8144 cell\r\nwww.avocadoit.com",
    "\r\n>\r\n> Ricardo\r\n> --------------------------\r\n> Ricardo Garcia\r\n> Sales Engineer\r\n>\r\n> AvocadoIT, Inc.\r\n> 2211 North First Street, Suite 200\r\n> San Jose, CA 95131\r\n> (408) 562-7984 (direct)\r\n> (408) 829-4827 (mobile)\r\n>\r\n> >\r\n> ",
    "\r\nLet me know,\r\nRyan\r\n____________________________\r\nRyan Glover \r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8057 office\r\n408.562.8100 fax\r\n408.420.3841 mobile\r\nryan.glover@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nHoward\r\n____________________________\r\nHoward Mora\r\nQA Director\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7954 (direct)\r\n(408) 562-8000 (main)\r\n(408) 562-8100 (fax)",
    "\r\nAlex Tran\r\nDirector of Product Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8068 office\r\n(408) 562-8100 fax\r\n(408) 420-3877 cell\r\nalex.tran@avocadoit.com",
    "\r\nMatt Moore\r\nSales Operations Specialist\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nDirect 408.562.8177  Fax 408.562.8101\r\nCell 408.857-8527\r\nMatt.Moore@avocadoit.com",
    "\r\nSincerely, Schemena\r\nDirector\r\nApplications Engineering\r\n<<...OLE_Obj...>>\r\nAvocadoIT Inc.\r\nTel.:  408.562.8028\r\nFAX: 408.562.8100\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemena@avocadoit.com <mailto:john.schemena@avocadoit.com>\r\n>  ",
    "\r\nMatt\r\nMatt DiMaria\r\nVP, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-8040 (direct)\r\n(408) 562-8000 (main)\r\nwww.avocadoit.com",
    "\r\nDebbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nTel: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\ndebbiemargulies@hotmail.com\r\nwww.avocadoit.com",
    "\r\n---  Garcia \r\nTechnical Marketing Engineer \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\n(408) 562-7984 office \r\n(408) 562-8100 fax \r\n(408) 829-4827 cell \r\nricardo.garcia@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nThanks in advance,\r\nMeg\r\nAndersen Consulting\r\nCIO - Solution Delivery\r\nBonnie H. Handmacher, Minneapolis, 27/78316 / (612) 277-8316\r\n11/16/2000 01:40 PM CST\r\nTo:   Margaret Travis@Andersen Consulting",
    "Doug Adkins\r\n> Director, Technical Support\r\n> AvocadoIT, Inc.\r\n> 2211 North First Street, Suite 200\r\n> San Jose, Ca. 95131\r\n> Office:(408) 562-7985\r\n> Mobile: (408) 829-5118\r\n> Fax: (408) 562-8100\r\n> Email: doug.adkins@avocadoit.com\r\n> \r\n>  <<...OLE_Obj...>>\r\n> www.avocadoit.com\r\n> \r\n>\r\n",
    "thanks!\r\nMeshele Ko\r\nDirector of Sales\r\n<< OLE Object: Picture (Metafile) >> \r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8034  \tMobile: 408.836.9665 \r\nMain: 408.562.8000 \tFax: 408.562.8100\r\nmko@avocadoit.com \r\nhttp://www.avocadoit.com\r\n",
    "\r\n>\r\n>Mark Tracy\r\n>\r\n>AvocadoIT, Inc.\r\n>2211 North First Street, Suite 200, San Jose, CA  95131\r\n>Phone: 408.562.8006 / Fax: 408. 588.0060\r\n>\r\n>\r\n>",
    "\r\nDavid Marcus\r\nDirector\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\ndirect (408) 562-7978\r\nmobile (650) 281-9010\r\nfax (408) 562-8100\r\ndavid.marcus@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nJennifer Park\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\ndirect: 408.562.8185\r\nwww.avocadoit.com",
    "\r\nNadir Nawabzada\r\nSystem Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nPhone: 408-562-8044\r\nCell: 408-205-7289\r\nEmail: nnawabzada@avocadoit.com",
    "\r\nRuth\r\nRuth Gantly\r\nManager, Information Products\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nemail ruth.gantly@avocadoit.com\r\nPhone: 408.562.8033\r\nFax: 408.562.8100",
    "\r\nGermana Martinez\r\nHuman Resources Coordinator\r\nand Executive Admin\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-8135 PH\r\n408-562-8107 FX",
    "\r\nBest Regards,\r\nBrett Welborn\r\nBrett W. Welborn\r\nAccount Development\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com \r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)\r\n",
    "\r\nMark\r\nMark Stewart \r\nFacility Manager \r\n408.562.8042 Direct & Fax \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nmark.stewart@avocadoit.com ",
    "\r\nRegards,\r\nMark Stewart \r\nFacility Manager \r\n408.562.8042 Direct & Fax \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nmark.stewart@avocadoit.com ",
    "\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 phone\r\n408.891.8144 cell\r\nwww.avocadoit.com",
    "\r\nTeresa M. Neves\r\nPayroll Administrator\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8196\r\nFax: 408.562.8100\r\nteresa.neves@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDonivan Fox\r\nManager, Applications Engineering\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8001\r\nMobile: 408.394.3613\r\nFax: 408.562.8100\r\ndonivan.fox@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks, Schemena\r\nDirector of Applications Engineering\r\n408.562.8028 Direct & Fax\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\njohn.schemenat@avocadoit.com <mail",
    "\r\nMatt Moore\r\nSales Operations Specialist\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, California 95131\r\nDirect 408.562.8177  Fax 408.562.8101\r\nCell 408-857-8527\r\nMatt.Moore@avocadoit.com",
    "Debbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com ",
    "Emma\r\nEmma L. Fowler\r\nSenior Channel Marketing Manager\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n408-562-7986\r\nemma.fowler@avocadoit.com <mail",
    "\r\nBrett Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95134\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "dbm\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n*New Address* \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com",
    "*********************************************\r\nT. Ty Wang\r\nBusiness Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-7945 Office\r\n(408) 562-8100 Fax\r\n*********************************************",
    "\r\nRegards\r\nDaimyo\r\nwww.avocadoit.com \r\nDaimyo Chen\r\nSystems Admin\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect:\t (408) 562-7968\r\nMobile:\t (408) 891-8138",
    "Debbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n*New Address*\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "Debbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nMain: 408.562.8000 \r\nDirect: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com ",
    "\r\nDebbie Margulies \r\nMarketing Manager \r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nTel: 408.562.8059 \r\nMobile: 408.393.1052 \r\nFax: 408.562.8100 \r\ndebbie.margulies@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nTnx,\r\nMatt\r\nMatt DiMaria\r\nVP, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-8040 (direct)",
    "Debbie Margulies\r\nMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8059\r\nMobile: 408.393.1052\r\nFax: 408.562.8100\r\ndebbie.margulies@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nBrett Welborn \r\nAccount Development Manager \r\nAvocadoIT, Inc. \r\nhttp://www.avocadoit.com \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA  95131 \r\n(800) 355-1103 x8186 \r\n(408) 562-8186 (direct) \r\n(408) 857-8524 (mobile) ",
    "\r\nDoug Adkins\r\nDirector, Technical Support\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, Ca. 95131\r\nOffice:(408) 562-7985\r\nMobile: (408) 821-6023\r\nFax: (408) 562-8100\r\nEmail: doug.adkins@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nRegards,\r\nMark H. Ricktor\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n(408) 562-8054 Office extension\r\n(650) 245-2624 Mobile",
    "\r\nTom\r\nThomas J. Stoddard \r\nDistrict Manager, West \r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200 \r\nSan Jose, CA 95131 \r\n949 425-9826 Direct\r\ntom.stoddard@avocadoit.com ",
    "\r\nKimberlie\r\nKimberlie L. Cerrone\r\nVice President and General Counsel\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA  95131\r\n408 562-8069 direct\r\n408 562-8109 fax\r\n408 910-5809 mobile\r\n44 7796148466 GSM \r\n650 344-9157 home\r\nkcerrone@avocadoit.com",
    "Thanks\r\nAlice Sun\r\nStaff Accountant\r\nAvocadoIT, Inc.\r\n2211 North First Street Suite200\r\nSan Jose, CA 95131\r\nDirect: 408-562-8048\r\nEmail: asun@avocadoit.com",
    "\r\nRoopak\r\nRoopak Parikh\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain:  408.562.8000\r\nDirect: 408.562.8023\r\nFax: 408.562.8100\r\nroopak.parikh@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nSusan\r\nSusan Simmons\r\nDirector of Finance & Administration\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8022 direct dial\r\n(408) 562-8000 corporate\r\n(408) 562-8100 fax\r\nsusan.simmons@avocadoit.com",
    "\r\nThanks,\r\nMark Stewart\r\nFacility Manager\r\n408.562.8042 Direct & Fax\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nmark.stewart@avocadoit.com",
    "\r\nThank you.\r\nMark Stewart\r\nFacility Manager\r\n408.562.8042 Direct & Fax\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nmark.stewart@avocadoit.com",
    "\r\nRegards,\r\nMichelle Cheng\r\nbea systems, inc.\r\n2315 North First Street\r\nSan Jose, CA  95131\r\nUSA\r\n+1-408-570-8050 (office)\r\n+1-253-276-4663 (efax)\r\nbea one of the world's leading e-business infrastructure software companies\r\nwww.bea.com\r\n",
    "\r\nThanks\r\njaime\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\nYour Business Everywhere!\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 ~ phone\r\n408.891.8144 ~ cell\r\nwww.avocadoit.com",
    "\r\njaime \r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\nYour Business Everywhere!\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 ~ phone\r\n408.891.8144 ~ cell\r\nwww.avocadoit.com",
    "\r\nT. Ty Wang\r\nAvocadoIT, Inc.\r\nBusiness Development\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7945 Office   (408) 562-8100 Fax\r\nty.wang@avocadoit.com",
    "\r\nAvocadoIT, Inc. \r\n2211 North First Street \r\nSuite 200 \r\nSan Jose, CA 95131 \r\nMain: 408.562.8000 \r\nFax: 408.562.8100 \r\nwww.avocadoit.com \r\nYour Business Everywhere!\r\n",
    'Representative Director\r\nAvocadoIT Japan KK\r\ntel: 408-205-9961 (cell/US), +81-90-4836-7219 (cell/Jpn)\r\ne-mail: toshiya.otani@avocadoit.com\r\nUS: 2211 North First Street Suite 200, San Jose, CA 95131\r\nJapan: 105-0001 4-3-20 Toranomon Minato-ku, Tokyo\r\nKamiyacho Mori Building 14F\r\nHP: www.avocadoit.com/jp\r\n"AvocadoIT, Your Business Everywhere!" \r\n***********************************************************',
    "\r\nPhani Syam Yadavalli\r\nAvocadoIT,Inc\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8074 (W)\r\n(408) 261-2602 (H)",
    "\r\nSr. Instructor\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7956\r\nMobile: \r\nFax: 408.562.8100 \r\ndouglas.field@avocadoit.com \r\nwww.avocadoit.com",
    "--------------------------\r\nSent To You Wirelessly From AvocadoIT (www.avocadoit.com)\r\nDavid Marcus\r\nDirector of Sales\r\nAvocadoIT, Inc .\r\n2211 North First Street\r\nSan Jose, CA  95131\r\nmobile (650) 281-9010\r\ndirect (408) 562-7978\r\ndavid.marcus@avocadoit.com",
    "\r\nPhil Gonzales\r\nFacilities Engineer\r\nAvocadoIT, Inc.\r\n2211North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\n(408) 562-8063 Direct\r\n(408) 205-7282 Mobile\r\n(408) 562-8100 Fax\r\nphil.gonzales@avocadoit.com",
    "Brett Welborn\r\nAccount Development Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "\r\nThanks,\r\nSusan\r\nSusan Simmons\r\nSenior Director of Finance and Corporate Controller\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000\r\nDirect: 408.562.8022\r\nMobile: 408.857-8738\r\nFax: 408.562.8100\r\nsusan.simmons@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks!\r\n-Lisa\r\nLisa Crowder\r\nExecutive Assistant to Prakash Iyer, C.T.O. and Founder\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\n2nd Floor\r\nSan Jose, CA  95131\r\n(408) 562-8012 - Direct\r\n(408) 562-8103 - Fax\r\nlisa.crowder@avocadoit.com\r\nwww.avocadoit.com\r\nAvocadoIT, your business everywhere!",
    "\r\nBithi Agrawal\r\nApplications Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8071\r\nFax: 408.562.8100\r\nbagrawal@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\n___________________________________________\r\nNilesh Bodade\r\nApplications Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8126\r\nFax: 408.562.8100\r\nnbodade@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nthanks,\r\nBIthi\r\nBithi Agrawal\r\nApplications Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nMain: 408.562.8000 \r\nDirect: 408.562.8071\r\nFax: 408.562.8100\r\nbagrawal@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThanks,\r\nTy\r\nT. Ty Wang\r\nDirector, Market Development\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7945\r\nFax: 408.562.8100\r\nty.wang@avocadoit.com\r\nwww.avocadoit.com\r\n",
    "\r\nDarshan\r\nDarshan Patel\r\nSenior Manager, Professional Services\r\nAvocadoIT, Inc.\r\n2211 North First Street \r\nSan Jose, CA 95131 \r\nDirect: 408.562.8024\r\ndarshan.patel@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nThank you all!\r\njaime\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\n2211 North First Street ~ Suite 200 • San Jose, CA  95131\r\n408.562.8015 ~ phone •  408.891.8144 ~ cell\r\nwww.avocadoit.com",
    "Iyer\r\nFounder & Chief Technology Officer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8030 \r\nMobile: 408.206.1058\r\nFax: 408.562.8100 \r\nprakash.iyer@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nthank you\r\njaime\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT, Inc.\r\nYour Business Everywhere!\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 ~ phone\r\n408.891.8144 ~ cell\r\nwww.avocadoit.com",
    "\r\nRitesh Adval \r\nAvocadoIT, Inc. \r\n2211 North First Street, \r\nSuite 200, San Jose, \r\nCA  95131 \r\nDirect: 408.562.8035 \r\nFax: 408.562.8100 \r\nradval@avocadoit.com \r\nwww.avocadoit.com",
    "\r\nMitch Stewart\r\nDirector of Client Development\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.8121\r\nMobile: 408.420-3865\r\nFax: 408.562.8102\r\nmitch.stewart@avocadoit.com\r\nwww.avocadoit.com",
    "\r\nDoug Adkins\r\nDirector, Technical Support\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, Ca. 95131\r\nOffice:(408) 562-7985\r\nMobile: (408) 829-5118\r\nFax: (408) 562-8100\r\nEmail: doug.adkins@avocadoit.com\r\nwww.avocadoit.com",
    "--- Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(408) 829-4827 cell\r\n(408) 562-7984 office\r\n(408) 562-8100 fax\r\nricardo.garcia@avocadoit.com\r\nwww.avocadoit.com",
    "Brett W. Welborn\r\nAccount Executive\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 ext. 8186\r\n(408) 562-8186 (direct)",
    "Miyuki Goldman\r\nQA Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, CA 95131\r\nDirect: 408.562.7979\r\nFax: 408.562.8100\r\nmiyuki.goldman@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "-------------------------- Garcia\r\nSales Engineer\r\nAvocadoIT, Inc.\r\n2211 North First Street, Suite 200\r\nSan Jose, CA 95131\r\n(408) 562-7984 (direct)\r\n(408) 829-4827 (mobile)",
    "Brett Welborn\r\nAccount Development Manager\r\nAvocadoIT, Inc.\r\nhttp://www.avocadoit.com\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n(800) 355-1103 x8186\r\n(408) 562-8186 (direct)\r\n(408) 857-8524 (mobile)",
    "\r\nMark Tracy\r\nAvocadoIT, Inc. \r\n2211 North First Street, Suite 200, San Jose, CA  95131\r\nPhone: 408.562.8006 / Fax: 408. 588.0060",
    "\r\nYolanda Vasquez\r\nFacilities Assistant\r\nAvocadoIT, Inc\r\n2211 North First Street\r\nSuite 200\r\nSan Jose, Ca 95131\r\nDirect: 408 562-8049\r\nMobile: 408 390-9268\r\nFax: 408 562-8100\r\nyolanda.vasquez@avocadoit.com\r\nwww.avocadoit.com <http://www.avocadoit.com>",
    "\r\njm\r\nJaime L. McNeely\r\neMarketing Manager\r\nAvocadoIT\r\n2211 North First Street, Suite 200\r\nSan Jose, CA  95131\r\n408.562.8015 phone\r\n408.891.8144 cell\r\nwww.avocadoit.com",
    "\r\nThanks!\r\nAndy\r\n--------------------------\r\nAndy Wong\r\nDirector, Marketing\r\nAvocadoIT, Inc.\r\n2211 North First Street\r\nSan Jose, CA 95131\r\nDirect: 408.562.8131\r\nFax: 408.562.8131\r\nawong@avocadoit.com\r\nwww.avocadoit.com",
    


    ]  # Regex patterns

for item in removal_list:
    texts = texts.str.replace(item,"")


In [ ]:
pattern = re.compile(pattern=r'\r\n----- Original Message -----\r\n.*',flags=re.DOTALL|re.MULTILINE)
texts = texts.apply(lambda x: re.sub(pattern,"", x))

texts = texts[~texts.str.contains('If you would like to stop or modify the delivery of Research via Email')]
texts = texts[~texts.str.contains('You are invited to join a meeting on the web, using WebEx.')]
texts = texts[~texts.str.contains('REGISTER TODAY')]
texts = texts[~texts.str.contains('-------------------------------------------------------------------------\r\nThis message has been sent to you electronically')]
texts = texts[~texts.str.startswith('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')]

# Remove Yahoo
pattern = re.compile(pattern=r'(?:\r\n)?\_+\r\nDo You Yahoo!\?\r\n.*?yahoo\.com.*?\r\n', flags=re.DOTALL | re.MULTILINE )
texts = texts.apply(lambda x: re.sub(pattern,"", x))


pt = r'^Return-Path: <.*?@.*?\.\w{2,3}>\r\n'
pt = re.compile(pt,flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^--=_alternative\s[A-F\d]{16}_=--(?:\r\n)?',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^--=_related\s[A-F\d]{16}_=.*?[A-F\d]{16}_=--\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^-{5}\sOriginal\sMessage\s-{5}(?:\s)?\r\n(?:From:.*?\r\nTo:.*?\r\nSubject:.*?\r\n)?',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:.*?Message-ID:.*?\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:.*?may not be legible.\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:.*?iso-8859-1"\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date:  [\/\d]{10}\s[\d:]{8}\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

pt = re.compile(r'^Date: .*?MIME-Version.*text\/plain\r\n',flags=re.DOTALL|re.MULTILINE | re.IGNORECASE)
texts = texts.apply(lambda x: re.sub(pt,"", x))

0         Bug 5539\r\nThe screen wizard now ignores the ...
1         At 02:50 PM 3/15/2001 +0100, it seemed like Ma...
2         All,\r\nWe are still in a Nor'easter, but the ...
3         Responding to Robert Sartin (07:02 AM 6/14/200...
4         I will try to get the real secure up at the fi...
                                ...                        
128640    Checked in WinInetBrowser.java to fix a bug in...
128641    Just wanted to make sure that you know I did t...
128642    In the end, you will always fail to think of e...
128643    Yes, it's valid for Europe, as well.  Venk for...
128644    Hey John,\r\nThe Siebel Adapter will allow you...
Name: text, Length: 128641, dtype: object

## Save to DF

In [11]:
df = texts.to_frame()
df.to_parquet('../../data/processed/05_avocado_cleaned_clustered_wip_7.parquet', engine='pyarrow')

In [ ]:
len(df)